In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_1(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_2(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_3(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_4(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
def NN_model_structure_regression_5(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [23]:
def NN_model_structure_regression_6(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [24]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [25]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_1(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 418us/step - lo

1052/1052 [==============================] - 0s 89us/step - loss: 22.1620 - val_loss: 26.7928
Epoch 66/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.9570 - val_loss: 25.3172
Epoch 67/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.2472 - val_loss: 25.4114
Epoch 68/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.8745 - val_loss: 25.5100
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.2060 - val_loss: 26.4694
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.5756 - val_loss: 26.3196
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1771 - val_loss: 26.2568
Epoch 72/1000
1052/1052 [==============================] - 0s 99us/step - loss: 21.9521 - val_loss: 25.9692
Epoch 73/1000
1052/1052 [==============================] - 0s 92us/step - loss: 21.8533 - val_loss: 25.5797
Epoch 74/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 83us/step - loss: 22.0295 - val_loss: 25.9537
Epoch 141/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.9681 - val_loss: 27.4006
Epoch 142/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.7482 - val_loss: 25.7903
Epoch 143/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.9564 - val_loss: 25.6262
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1678 - val_loss: 26.7575
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.1091 - val_loss: 27.0167
Epoch 146/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.8499 - val_loss: 25.4047
Epoch 147/1000
1052/1052 [==============================] - 0s 91us/step - loss: 22.3900 - val_loss: 25.5879
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.7447 - val_loss: 25.8411
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 93us/step - loss: 17.9116 - val_loss: 20.6253
Epoch 216/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.8488 - val_loss: 20.9252
Epoch 217/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.6877 - val_loss: 20.5600
Epoch 218/1000
1052/1052 [==============================] - 0s 86us/step - loss: 18.0774 - val_loss: 21.3712
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.2449 - val_loss: 20.3562
Epoch 220/1000
1052/1052 [==============================] - 0s 85us/step - loss: 17.2031 - val_loss: 20.6044
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.3925 - val_loss: 22.3830
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.3409 - val_loss: 19.7210
Epoch 223/1000
1052/1052 [==============================] - 0s 94us/step - loss: 17.1068 - val_loss: 19.9370
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 91us/step - loss: 11.9856 - val_loss: 14.8705
Epoch 291/1000
1052/1052 [==============================] - 0s 93us/step - loss: 12.2136 - val_loss: 16.0879
Epoch 292/1000
1052/1052 [==============================] - 0s 91us/step - loss: 12.4297 - val_loss: 15.2907
Epoch 293/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.3315 - val_loss: 15.5563
Epoch 294/1000
1052/1052 [==============================] - 0s 83us/step - loss: 12.4003 - val_loss: 15.9444
Epoch 295/1000
1052/1052 [==============================] - 0s 82us/step - loss: 12.1429 - val_loss: 14.4451
Epoch 296/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.4516 - val_loss: 15.6875
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.5920 - val_loss: 17.0516
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.0080 - val_loss: 15.2817
Epoch 299/1000
1052/1052 [========

...

Epoch 745/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8618 - val_loss: 10.9666
Epoch 746/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0664 - val_loss: 11.7266
Epoch 747/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.0457 - val_loss: 11.1659
Epoch 748/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.2393 - val_loss: 10.5296
Epoch 749/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9308 - val_loss: 10.4935
Epoch 750/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1356 - val_loss: 11.2305
Epoch 751/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.3260 - val_loss: 12.1757
Epoch 752/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.2201 - val_loss: 10.7167
Epoch 753/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7640 - val_loss: 10.8122
Epoch 754/1000
1052/1052 [==

Epoch 821/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2534 - val_loss: 10.6141
Epoch 822/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9088 - val_loss: 10.4848
Epoch 823/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9322 - val_loss: 11.1810
Epoch 824/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0754 - val_loss: 10.7780
Epoch 825/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7747 - val_loss: 11.1674
Epoch 826/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.7940 - val_loss: 10.7899
Epoch 827/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.0669 - val_loss: 11.6913
Epoch 828/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.7825 - val_loss: 10.7337
Epoch 829/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.6826 - val_loss: 10.6795
Epoch 830/1000
1052/1052 [==

Epoch 897/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0179 - val_loss: 11.0638
Epoch 898/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.3357 - val_loss: 11.5366
Epoch 899/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1380 - val_loss: 10.4415
Epoch 900/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8946 - val_loss: 10.6532
Epoch 901/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.5577 - val_loss: 10.3589
Epoch 902/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8486 - val_loss: 10.4589
Epoch 903/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0346 - val_loss: 9.9388
Epoch 904/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5843 - val_loss: 10.4917
Epoch 905/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6255 - val_loss: 10.7084
Epoch 906/1000
1052/1052 [===

Epoch 973/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6753 - val_loss: 10.6563
Epoch 974/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7951 - val_loss: 10.3352
Epoch 975/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6902 - val_loss: 11.4051
Epoch 976/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.8574 - val_loss: 10.1648
Epoch 977/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.7651 - val_loss: 10.0015
Epoch 978/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6447 - val_loss: 11.4058
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7364 - val_loss: 10.3301
Epoch 980/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7514 - val_loss: 11.6658
Epoch 981/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8862 - val_loss: 11.1894
Epoch 982/1000
1052/1052 [==

[array([[-1.1028473 , -0.18292013, -3.8349018 , -1.5870104 ,  3.814878  ],
        [-0.13712819, -0.2612418 ,  0.32714096,  0.35182697, -0.08928601],
        [-0.3918271 , -0.34844434, -0.5450464 ,  0.28817612, -0.074275  ],
        [ 0.10039558,  0.09183448,  0.16906567,  0.07121518, -0.07979812],
        [-1.0957295 , -0.21084051, -0.45275334, -0.18003482,  2.3101661 ]],
       dtype=float32),
 array([-3.041849  , -0.20409858, -4.772107  , -1.2570957 ,  5.236213  ],
       dtype=float32),
 array([[ 2.8811598 ,  1.933249  , -2.2385168 ,  1.9186612 , -2.9452496 ],
        [-1.7918831 , -1.0472606 ,  0.5667873 , -1.836486  ,  1.3318132 ],
        [ 3.2278888 ,  2.5148187 , -2.7223623 ,  2.6507337 , -3.2211773 ],
        [-1.4626925 , -0.67815995,  1.6746815 , -0.9251087 ,  0.37785146],
        [-1.8383002 , -2.7416701 ,  2.0179338 , -2.8827069 ,  1.9486701 ]],
       dtype=float32),
 array([-2.3521082, -2.5822237,  2.24171  , -2.1694663,  2.1959887],
       dtype=float32),
 array([[-2.5

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure1_5th.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_2(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 200us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.2089 - val_loss: 19.5075
Epoch 68/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.1242 - val_loss: 19.3416
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 15.8350 - val_loss: 19.9958
Epoch 70/1000
1052/1052 [==============================] - 0s 93us/step - loss: 16.1248 - val_loss: 19.3492
Epoch 71/1000
1052/1052 [==============================] - 0s 91us/step - loss: 15.8564 - val_loss: 19.3934
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.7702 - val_loss: 19.3287
Epoch 73/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.9922 - val_loss: 19.0708
Epoch 74/1000
1052/1052 [==============================] - 0s 95us/step - loss: 15.5568 - val_loss: 18.9046
Epoch 75/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.5847 - val_loss: 18.9612
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 93us/step - loss: 11.5284 - val_loss: 14.9906
Epoch 143/1000
1052/1052 [==============================] - 0s 91us/step - loss: 12.0965 - val_loss: 15.4005
Epoch 144/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.9670 - val_loss: 15.9512
Epoch 145/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.7337 - val_loss: 15.4522
Epoch 146/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.3155 - val_loss: 15.1765
Epoch 147/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.0572 - val_loss: 13.3775
Epoch 148/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.3576 - val_loss: 14.8059
Epoch 149/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.4022 - val_loss: 14.4417
Epoch 150/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.1558 - val_loss: 13.9098
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 92us/step - loss: 8.9282 - val_loss: 11.4515
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.5887 - val_loss: 11.0881
Epoch 219/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.5339 - val_loss: 11.8305
Epoch 220/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.2742 - val_loss: 11.3637
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.2541 - val_loss: 11.6974
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1088 - val_loss: 11.1504
Epoch 223/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1133 - val_loss: 12.6126
Epoch 224/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.1025 - val_loss: 14.4692
Epoch 225/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2018 - val_loss: 11.3706
Epoch 226/1000
1052/1052 [================

Epoch 293/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.8695 - val_loss: 11.6694
Epoch 294/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.9787 - val_loss: 11.0406
Epoch 295/1000
1052/1052 [==============================] - 0s 113us/step - loss: 8.7906 - val_loss: 11.3081
Epoch 296/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.6202 - val_loss: 10.1370
Epoch 297/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.8161 - val_loss: 10.2948
Epoch 298/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4550 - val_loss: 12.3072
Epoch 299/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.6703 - val_loss: 11.8946
Epoch 300/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7988 - val_loss: 10.6615
Epoch 301/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.6494 - val_loss: 12.4365
Epoch 302/1000
1052/1052 [=

...

1052/1052 [==============================] - 0s 89us/step - loss: 8.1442 - val_loss: 9.5745
Epoch 750/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9472 - val_loss: 9.3691
Epoch 751/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.3311 - val_loss: 9.5828
Epoch 752/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.3583 - val_loss: 9.4184
Epoch 753/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4324 - val_loss: 10.5045
Epoch 754/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0602 - val_loss: 9.2007
Epoch 755/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.2558 - val_loss: 10.4497
Epoch 756/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3087 - val_loss: 10.7226
Epoch 757/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.0889 - val_loss: 9.1242
Epoch 758/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 90us/step - loss: 8.4505 - val_loss: 9.1828
Epoch 826/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4558 - val_loss: 10.2113
Epoch 827/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.2693 - val_loss: 9.1801
Epoch 828/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.9276 - val_loss: 9.7640
Epoch 829/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2973 - val_loss: 11.4217
Epoch 830/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9439 - val_loss: 9.4929
Epoch 831/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1451 - val_loss: 9.6973
Epoch 832/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4184 - val_loss: 10.0119
Epoch 833/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4431 - val_loss: 9.3794
Epoch 834/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 87us/step - loss: 8.3787 - val_loss: 9.9284
Epoch 902/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2148 - val_loss: 9.3546
Epoch 903/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1507 - val_loss: 9.8372
Epoch 904/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.0400 - val_loss: 9.6574
Epoch 905/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.5380 - val_loss: 9.4051
Epoch 906/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.7230 - val_loss: 9.6156
Epoch 907/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2375 - val_loss: 9.1159
Epoch 908/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.7191 - val_loss: 9.9844
Epoch 909/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.6392 - val_loss: 10.3952
Epoch 910/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 92us/step - loss: 8.2019 - val_loss: 8.9538
Epoch 978/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.3417 - val_loss: 11.4980
Epoch 979/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4635 - val_loss: 8.8463
Epoch 980/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6397 - val_loss: 10.1211
Epoch 981/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.7474 - val_loss: 10.2284
Epoch 982/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4928 - val_loss: 9.2731
Epoch 983/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.9202 - val_loss: 9.6228
Epoch 984/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.0156 - val_loss: 9.6599
Epoch 985/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.0822 - val_loss: 9.8731
Epoch 986/1000
1052/1052 [=======================

[array([[-0.7677883 ,  3.3786387 ,  1.8246051 ,  0.1774328 , -4.090878  ],
        [-1.5376334 , -0.1165167 , -0.49760297,  0.1743749 ,  0.23387308],
        [-1.6489935 , -0.01073588, -0.422408  ,  0.26362455, -0.7587081 ],
        [ 0.27092138, -0.05603147, -0.01877734, -0.12452704,  0.1712811 ],
        [ 0.38842356,  2.4592583 ,  0.30092913,  0.14813006, -0.33776888]],
       dtype=float32),
 array([-0.71774656,  4.8482175 ,  1.3255492 , -1.3909711 , -4.9423018 ],
       dtype=float32),
 array([[-0.49358094, -0.80332774, -0.6153615 , -0.5802122 ,  0.16576518,
          0.5420509 , -0.21803573, -0.27171144,  0.30409363, -0.46186498],
        [-1.7936229 , -1.1690708 , -1.2502135 , -1.2135376 ,  1.396115  ,
          1.8445383 ,  0.7554351 , -1.9695513 , -2.0240016 , -2.0544276 ],
        [ 0.69083965,  1.0306183 ,  0.16364534,  0.28036234, -1.1024816 ,
          0.00622307, -0.7980508 ,  0.7393603 ,  0.57989544,  1.0012525 ],
        [ 1.1746765 ,  1.8081229 ,  1.8896163 ,  1.840676

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure2_5th.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_3(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                90        
_________________________________________________________________
activation_6 (Activation)    (None, 15)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 136
Trainable params: 136
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 207us/step - l

1052/1052 [==============================] - 0s 86us/step - loss: 18.1843 - val_loss: 22.7289
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.7216 - val_loss: 23.1541
Epoch 69/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.6177 - val_loss: 22.7922
Epoch 70/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.2479 - val_loss: 22.9738
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.8131 - val_loss: 21.7531
Epoch 72/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.3771 - val_loss: 21.3585
Epoch 73/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.1085 - val_loss: 20.9554
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.9563 - val_loss: 21.0343
Epoch 75/1000
1052/1052 [==============================] - 0s 93us/step - loss: 17.4295 - val_loss: 20.7376
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 91us/step - loss: 11.7576 - val_loss: 15.1400
Epoch 143/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.6885 - val_loss: 14.6653
Epoch 144/1000
1052/1052 [==============================] - 0s 90us/step - loss: 12.2718 - val_loss: 16.5572
Epoch 145/1000
1052/1052 [==============================] - 0s 93us/step - loss: 12.3299 - val_loss: 15.7089
Epoch 146/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.9266 - val_loss: 15.4064
Epoch 147/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.2261 - val_loss: 14.6092
Epoch 148/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.9968 - val_loss: 15.1510
Epoch 149/1000
1052/1052 [==============================] - 0s 93us/step - loss: 12.1297 - val_loss: 16.6617
Epoch 150/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.1237 - val_loss: 14.9859
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 95us/step - loss: 11.8273 - val_loss: 13.1882
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.4429 - val_loss: 12.4635
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.1425 - val_loss: 12.5670
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.1676 - val_loss: 12.2626
Epoch 221/1000
1052/1052 [==============================] - 0s 92us/step - loss: 10.4106 - val_loss: 12.6423
Epoch 222/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.2805 - val_loss: 14.1350
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.6726 - val_loss: 12.3569
Epoch 224/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.0013 - val_loss: 12.2160
Epoch 225/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.8769 - val_loss: 12.3370
Epoch 226/1000
1052/1052 [=========

1052/1052 [==============================] - 0s 89us/step - loss: 8.8969 - val_loss: 10.6155
Epoch 293/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.5505 - val_loss: 11.1827
Epoch 294/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7440 - val_loss: 10.8157
Epoch 295/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7084 - val_loss: 11.6913
Epoch 296/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.3142 - val_loss: 11.8004
Epoch 297/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.5012 - val_loss: 11.6024
Epoch 298/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.1396 - val_loss: 10.8516
Epoch 299/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.5360 - val_loss: 11.3700
Epoch 300/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.0050 - val_loss: 9.9258
Epoch 301/1000
1052/1052 [==================

...

Epoch 748/1000
1052/1052 [==============================] - 0s 119us/step - loss: 7.2463 - val_loss: 8.3448
Epoch 749/1000
1052/1052 [==============================] - 0s 111us/step - loss: 8.5172 - val_loss: 10.4147
Epoch 750/1000
1052/1052 [==============================] - 0s 110us/step - loss: 7.5253 - val_loss: 8.6133
Epoch 751/1000
1052/1052 [==============================] - 0s 105us/step - loss: 7.5140 - val_loss: 8.2844
Epoch 752/1000
1052/1052 [==============================] - 0s 107us/step - loss: 7.3435 - val_loss: 8.8620
Epoch 753/1000
1052/1052 [==============================] - 0s 104us/step - loss: 7.6129 - val_loss: 9.0153
Epoch 754/1000
1052/1052 [==============================] - 0s 110us/step - loss: 7.4127 - val_loss: 9.7274
Epoch 755/1000
1052/1052 [==============================] - 0s 107us/step - loss: 7.5087 - val_loss: 9.7952
Epoch 756/1000
1052/1052 [==============================] - 0s 102us/step - loss: 7.7269 - val_loss: 8.4787
Epoch 757/1000
1052/1052 [=

1052/1052 [==============================] - 0s 97us/step - loss: 7.4117 - val_loss: 8.5260
Epoch 825/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.4787 - val_loss: 9.2301
Epoch 826/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.6561 - val_loss: 8.7214
Epoch 827/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.4885 - val_loss: 8.4450
Epoch 828/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.8054 - val_loss: 8.3886
Epoch 829/1000
1052/1052 [==============================] - 0s 99us/step - loss: 7.4378 - val_loss: 9.2095
Epoch 830/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.6798 - val_loss: 9.9837
Epoch 831/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.4680 - val_loss: 8.8883
Epoch 832/1000
1052/1052 [==============================] - 0s 96us/step - loss: 7.5110 - val_loss: 8.5835
Epoch 833/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 98us/step - loss: 7.8042 - val_loss: 9.2207
Epoch 901/1000
1052/1052 [==============================] - 0s 102us/step - loss: 7.5773 - val_loss: 9.2835
Epoch 902/1000
1052/1052 [==============================] - 0s 100us/step - loss: 7.4531 - val_loss: 9.0779
Epoch 903/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.5015 - val_loss: 8.4642
Epoch 904/1000
1052/1052 [==============================] - 0s 102us/step - loss: 7.6467 - val_loss: 10.5129
Epoch 905/1000
1052/1052 [==============================] - 0s 99us/step - loss: 7.5275 - val_loss: 9.4706
Epoch 906/1000
1052/1052 [==============================] - 0s 98us/step - loss: 7.8789 - val_loss: 9.3211
Epoch 907/1000
1052/1052 [==============================] - 0s 101us/step - loss: 7.4020 - val_loss: 8.3284
Epoch 908/1000
1052/1052 [==============================] - 0s 98us/step - loss: 7.8567 - val_loss: 8.8059
Epoch 909/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 104us/step - loss: 8.1814 - val_loss: 8.6680
Epoch 977/1000
1052/1052 [==============================] - 0s 100us/step - loss: 7.5533 - val_loss: 8.6027
Epoch 978/1000
1052/1052 [==============================] - 0s 97us/step - loss: 7.6335 - val_loss: 8.4866
Epoch 979/1000
1052/1052 [==============================] - 0s 105us/step - loss: 7.7869 - val_loss: 8.7380
Epoch 980/1000
1052/1052 [==============================] - 0s 104us/step - loss: 7.4681 - val_loss: 8.8813
Epoch 981/1000
1052/1052 [==============================] - 0s 114us/step - loss: 8.5728 - val_loss: 10.8663
Epoch 982/1000
1052/1052 [==============================] - 0s 112us/step - loss: 8.2116 - val_loss: 9.0210
Epoch 983/1000
1052/1052 [==============================] - 0s 107us/step - loss: 7.7486 - val_loss: 9.4328
Epoch 984/1000
1052/1052 [==============================] - 0s 106us/step - loss: 7.9881 - val_loss: 8.6522
Epoch 985/1000
1052/1052 [=================

[array([[-2.7919672 ,  2.1022475 , -0.17914449, -3.173222  , -0.5609557 ],
        [ 0.25407743,  0.09484929, -0.2519919 ,  0.15588671, -0.26156175],
        [ 0.15325333,  0.0568601 , -0.62885857, -0.97730666,  0.60064954],
        [ 0.12000665, -0.08048029,  0.09868743,  0.13511463,  0.0538561 ],
        [-0.25885865,  1.4075347 , -0.10699441, -0.157861  , -1.0371504 ]],
       dtype=float32),
 array([-1.5383961,  3.1506953, -0.2629808, -4.4808993, -1.0722444],
       dtype=float32),
 array([[-0.09233887,  0.481585  , -0.36302447,  0.00980702, -0.40432152,
          0.37571558, -0.654326  ,  0.4271522 ,  0.23243321, -0.39100522,
         -0.42023462,  0.6639396 , -0.8765567 ,  0.39006215, -0.28157026],
        [-1.0166435 ,  1.8106722 , -2.1201212 , -1.1487098 , -2.1342962 ,
          1.4021574 , -1.7075852 ,  1.2510626 ,  1.784408  , -1.3969655 ,
         -1.4233822 ,  1.5004185 , -2.2376816 ,  1.9360279 , -1.6135662 ],
        [-0.18518306,  0.35424858, -1.0290703 , -0.53976727, -0

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure3_5th.h5')

In [32]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_4(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_8 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 271us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 93us/step - loss: 14.8016 - val_loss: 17.1959
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.3545 - val_loss: 17.5371
Epoch 69/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.1172 - val_loss: 18.5883
Epoch 70/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.6832 - val_loss: 16.7790
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.8717 - val_loss: 17.2605
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.9937 - val_loss: 16.6543
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.0193 - val_loss: 17.0900
Epoch 74/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.9492 - val_loss: 16.6109
Epoch 75/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.0715 - val_loss: 18.9898
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 102us/step - loss: 9.0468 - val_loss: 10.5769
Epoch 143/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.5358 - val_loss: 11.8217
Epoch 144/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.4388 - val_loss: 10.6753
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1331 - val_loss: 10.9796
Epoch 146/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.2609 - val_loss: 14.0050
Epoch 147/1000
1052/1052 [==============================] - 0s 101us/step - loss: 9.7291 - val_loss: 10.8654
Epoch 148/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.6340 - val_loss: 11.4448
Epoch 149/1000
1052/1052 [==============================] - 0s 100us/step - loss: 9.0129 - val_loss: 10.7142
Epoch 150/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.9461 - val_loss: 11.7913
Epoch 151/1000
1052/1052 [=============

Epoch 218/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.3345 - val_loss: 10.9767
Epoch 219/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.8747 - val_loss: 10.1683
Epoch 220/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.7544 - val_loss: 12.7143
Epoch 221/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.4841 - val_loss: 9.9189
Epoch 222/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.4391 - val_loss: 9.2719
Epoch 223/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.5362 - val_loss: 10.2245
Epoch 224/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.5384 - val_loss: 9.1879
Epoch 225/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.4060 - val_loss: 9.5169
Epoch 226/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.3757 - val_loss: 11.2890
Epoch 227/1000
1052/1052 [====

1052/1052 [==============================] - 0s 104us/step - loss: 8.7331 - val_loss: 8.6574
Epoch 295/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.2278 - val_loss: 8.8947
Epoch 296/1000
1052/1052 [==============================] - 0s 106us/step - loss: 7.8008 - val_loss: 8.5826
Epoch 297/1000
1052/1052 [==============================] - 0s 104us/step - loss: 7.7309 - val_loss: 9.7491
Epoch 298/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.9939 - val_loss: 9.0477
Epoch 299/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.9800 - val_loss: 9.0669
Epoch 300/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9495 - val_loss: 9.1045
Epoch 301/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.9753 - val_loss: 8.5519
Epoch 302/1000
1052/1052 [==============================] - 0s 102us/step - loss: 7.9961 - val_loss: 9.4728
Epoch 303/1000
1052/1052 [======================

...

1052/1052 [==============================] - 0s 87us/step - loss: 5.6640 - val_loss: 5.9682
Epoch 751/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.2768 - val_loss: 5.7192
Epoch 752/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.9467 - val_loss: 5.6364
Epoch 753/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.2271 - val_loss: 5.7632
Epoch 754/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3973 - val_loss: 5.7137
Epoch 755/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.3436 - val_loss: 6.2496
Epoch 756/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.6521 - val_loss: 5.6294
Epoch 757/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.4230 - val_loss: 6.2438
Epoch 758/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.0556 - val_loss: 5.8860
Epoch 759/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 106us/step - loss: 5.3793 - val_loss: 5.5304
Epoch 827/1000
1052/1052 [==============================] - 0s 107us/step - loss: 5.5942 - val_loss: 5.6998
Epoch 828/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.1631 - val_loss: 5.9722
Epoch 829/1000
1052/1052 [==============================] - 0s 100us/step - loss: 5.4127 - val_loss: 5.6684
Epoch 830/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.5884 - val_loss: 6.1954
Epoch 831/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.5132 - val_loss: 5.5394
Epoch 832/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.3598 - val_loss: 5.7114
Epoch 833/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.5755 - val_loss: 6.7227
Epoch 834/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.4682 - val_loss: 5.8544
Epoch 835/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 90us/step - loss: 5.4539 - val_loss: 6.2745
Epoch 903/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.3615 - val_loss: 5.9619
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.0660 - val_loss: 5.6818
Epoch 905/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.2269 - val_loss: 5.3865
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.4698 - val_loss: 5.5678
Epoch 907/1000
1052/1052 [==============================] - 0s 94us/step - loss: 5.1840 - val_loss: 6.7496
Epoch 908/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.4023 - val_loss: 5.2171
Epoch 909/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.2568 - val_loss: 5.8946
Epoch 910/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.5781 - val_loss: 5.4093
Epoch 911/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.2875 - val_loss: 5.8580
Epoch 979/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.1959 - val_loss: 5.4944
Epoch 980/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.1820 - val_loss: 5.6033
Epoch 981/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.0598 - val_loss: 5.4278
Epoch 982/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.2365 - val_loss: 5.7157
Epoch 983/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.1728 - val_loss: 6.0042
Epoch 984/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.2118 - val_loss: 5.4785
Epoch 985/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.3036 - val_loss: 5.8174
Epoch 986/1000
1052/1052 [==============================] - 0s 106us/step - loss: 5.3772 - val_loss: 7.1044
Epoch 987/1000
1052/1052 [=========================

[array([[-3.8788285 , -3.4316714 , -0.40708408, -0.158925  , -0.8343887 ,
          0.12650158,  5.272518  , -4.6384478 , -0.4946385 , -4.1706696 ],
        [ 0.33565846,  0.21593846, -0.4197381 , -0.27487323, -1.2772856 ,
          0.14677253, -0.36620855, -0.81526196, -0.7610436 , -0.2665006 ],
        [ 0.09765035, -0.10054585, -0.49387273, -1.823727  , -0.3966244 ,
          0.16601782,  0.74108666, -3.4887586 , -2.2688293 , -0.05067521],
        [ 0.11059216,  0.0581785 ,  0.67049825,  0.07663258,  0.3054783 ,
         -0.10806639, -0.21898621,  0.24698038, -0.73153603, -0.3839056 ],
        [-3.536333  , -0.4103085 , -0.3436026 , -0.01880508,  0.13174582,
          0.13959658,  0.30670133, -0.28983793, -1.1406373 , -0.09629481]],
       dtype=float32),
 array([-5.289814 , -1.8584602,  2.8561058, -1.4011571,  0.1072287,
        -4.7049546,  6.2568574, -5.4900203,  6.7401867, -4.943392 ],
       dtype=float32),
 array([[ 2.0303774 ,  1.42294   , -1.8294666 ,  2.1805906 , -1.5242388

In [33]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure4_5th.h5')

In [34]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_5(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 238us/step - l

1052/1052 [==============================] - 0s 93us/step - loss: 11.7095 - val_loss: 14.7149
Epoch 68/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.8009 - val_loss: 14.8064
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.6260 - val_loss: 14.1438
Epoch 70/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.7583 - val_loss: 14.8345
Epoch 71/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.7645 - val_loss: 14.5577
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.5882 - val_loss: 13.8161
Epoch 73/1000
1052/1052 [==============================] - 0s 92us/step - loss: 12.0859 - val_loss: 13.6802
Epoch 74/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.6901 - val_loss: 14.5072
Epoch 75/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.6655 - val_loss: 13.7796
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 89us/step - loss: 9.8763 - val_loss: 12.5846
Epoch 143/1000
1052/1052 [==============================] - 0s 105us/step - loss: 9.9826 - val_loss: 11.9624
Epoch 144/1000
1052/1052 [==============================] - 0s 91us/step - loss: 10.2659 - val_loss: 12.3516
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.2281 - val_loss: 12.2626
Epoch 146/1000
1052/1052 [==============================] - 0s 93us/step - loss: 10.4506 - val_loss: 12.3083
Epoch 147/1000
1052/1052 [==============================] - 0s 104us/step - loss: 10.0585 - val_loss: 12.4940
Epoch 148/1000
1052/1052 [==============================] - 0s 98us/step - loss: 11.0458 - val_loss: 13.2706
Epoch 149/1000
1052/1052 [==============================] - 0s 99us/step - loss: 10.3184 - val_loss: 12.3042
Epoch 150/1000
1052/1052 [==============================] - 0s 94us/step - loss: 10.4121 - val_loss: 11.6312
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 94us/step - loss: 9.8621 - val_loss: 11.7429
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.5484 - val_loss: 11.6354
Epoch 219/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.5902 - val_loss: 11.7168
Epoch 220/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.9903 - val_loss: 11.0953
Epoch 221/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.4327 - val_loss: 15.9787
Epoch 222/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.2817 - val_loss: 11.0059
Epoch 223/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.6060 - val_loss: 11.0498
Epoch 224/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2142 - val_loss: 11.1176
Epoch 225/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.8332 - val_loss: 11.8988
Epoch 226/1000
1052/1052 [================

Epoch 293/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9815 - val_loss: 10.3232
Epoch 294/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0984 - val_loss: 10.8483
Epoch 295/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.8233 - val_loss: 11.2258
Epoch 296/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2386 - val_loss: 10.5179
Epoch 297/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9391 - val_loss: 9.9734
Epoch 298/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3930 - val_loss: 10.1805
Epoch 299/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.3429 - val_loss: 12.0277
Epoch 300/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.6542 - val_loss: 11.9974
Epoch 301/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5079 - val_loss: 10.7628
Epoch 302/1000
1052/1052 [===

...

1052/1052 [==============================] - 0s 94us/step - loss: 6.0809 - val_loss: 7.5936
Epoch 750/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.2669 - val_loss: 6.0960
Epoch 751/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.7487 - val_loss: 6.1851
Epoch 752/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.6696 - val_loss: 6.2178
Epoch 753/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.8855 - val_loss: 6.1326
Epoch 754/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.0138 - val_loss: 7.1781
Epoch 755/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.8862 - val_loss: 6.9515
Epoch 756/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.7018 - val_loss: 6.9224
Epoch 757/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.0425 - val_loss: 6.5548
Epoch 758/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 90us/step - loss: 5.6703 - val_loss: 6.8792
Epoch 826/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5656 - val_loss: 6.3371
Epoch 827/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.5194 - val_loss: 7.3955
Epoch 828/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.6866 - val_loss: 6.0150
Epoch 829/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5266 - val_loss: 7.1975
Epoch 830/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.5403 - val_loss: 5.4030
Epoch 831/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.4992 - val_loss: 5.9178
Epoch 832/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.4186 - val_loss: 6.9774
Epoch 833/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.4881 - val_loss: 6.1933
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.3341 - val_loss: 6.8457
Epoch 902/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.6217 - val_loss: 6.0129
Epoch 903/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.6009 - val_loss: 5.9833
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.5593 - val_loss: 5.2343
Epoch 905/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.8456 - val_loss: 6.2015
Epoch 906/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.3897 - val_loss: 5.5518
Epoch 907/1000
1052/1052 [==============================] - 0s 98us/step - loss: 5.3835 - val_loss: 5.4066
Epoch 908/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.4156 - val_loss: 6.2510
Epoch 909/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.5316 - val_loss: 5.2734
Epoch 910/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.4564 - val_loss: 6.1120
Epoch 978/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.3532 - val_loss: 5.3122
Epoch 979/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.5566 - val_loss: 6.4867
Epoch 980/1000
1052/1052 [==============================] - 0s 101us/step - loss: 5.3192 - val_loss: 5.2080
Epoch 981/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.3375 - val_loss: 5.8007
Epoch 982/1000
1052/1052 [==============================] - 0s 94us/step - loss: 5.7328 - val_loss: 5.6107
Epoch 983/1000
1052/1052 [==============================] - 0s 94us/step - loss: 5.6011 - val_loss: 5.6486
Epoch 984/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.2999 - val_loss: 5.4324
Epoch 985/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.3728 - val_loss: 6.9756
Epoch 986/1000
1052/1052 [=========================

[array([[ 0.5114426 ,  3.7540421 , -0.9258749 ,  3.9511173 , -2.8780782 ,
          3.5148356 , -0.17075485, -0.21496631,  0.5896957 ,  0.37322915],
        [ 1.1511277 ,  0.5803845 , -0.8427422 , -0.29231417,  0.13460119,
         -0.10889622,  0.11545078, -0.7047312 ,  0.32383096, -0.5949959 ],
        [ 1.8395011 ,  2.544752  , -0.34529173,  0.6556465 , -0.07748242,
         -0.10527457, -0.41793054, -1.9265183 ,  0.5396208 , -2.7228565 ],
        [ 0.26404285, -0.1661447 ,  0.3451975 , -0.1278867 ,  0.04879194,
         -0.03528875,  0.27694777,  0.09411192, -0.8269575 , -0.17434831],
        [ 0.64437515,  0.31596923, -1.2869954 ,  0.22923848, -0.24794689,
          2.180064  , -1.2812476 ,  0.87846226,  0.8659223 ,  3.088846  ]],
       dtype=float32),
 array([-5.344633  ,  4.3078933 , -0.37254304,  5.1133947 , -1.6516544 ,
         4.539762  , -3.7096655 , -1.0651788 , -2.757848  , -5.13457   ],
       dtype=float32),
 array([[ 0.9656323 ,  1.4725482 , -0.79799885,  1.4379953 , 

In [35]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure5_5th.h5')

In [36]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_6(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_12 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 249us/step - l

1052/1052 [==============================] - 0s 95us/step - loss: 9.6231 - val_loss: 11.1200
Epoch 68/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.4627 - val_loss: 11.4531
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.6741 - val_loss: 11.3956
Epoch 70/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.5182 - val_loss: 11.0879
Epoch 71/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.7236 - val_loss: 12.1863
Epoch 72/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.3603 - val_loss: 11.8632
Epoch 73/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.6664 - val_loss: 11.7720
Epoch 74/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.3992 - val_loss: 12.8155
Epoch 75/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.8723 - val_loss: 10.6586
Epoch 76/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 96us/step - loss: 8.7575 - val_loss: 9.5200
Epoch 144/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.5125 - val_loss: 8.9030
Epoch 145/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2299 - val_loss: 8.8288
Epoch 146/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2586 - val_loss: 9.6291
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3638 - val_loss: 9.8440
Epoch 148/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.1631 - val_loss: 10.1855
Epoch 149/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.6496 - val_loss: 9.1454
Epoch 150/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.8125 - val_loss: 9.1113
Epoch 151/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.7294 - val_loss: 9.4208
Epoch 152/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 89us/step - loss: 6.9529 - val_loss: 7.5092
Epoch 220/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.2339 - val_loss: 7.4068
Epoch 221/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.5966 - val_loss: 9.9303
Epoch 222/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.6982 - val_loss: 7.5926
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.8770 - val_loss: 7.6803
Epoch 224/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.7914 - val_loss: 7.7218
Epoch 225/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.9878 - val_loss: 8.4586
Epoch 226/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.3646 - val_loss: 8.5439
Epoch 227/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.5384 - val_loss: 8.5527
Epoch 228/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 92us/step - loss: 6.8134 - val_loss: 8.1978
Epoch 296/1000
1052/1052 [==============================] - 0s 103us/step - loss: 6.2721 - val_loss: 8.0264
Epoch 297/1000
1052/1052 [==============================] - 0s 99us/step - loss: 6.2424 - val_loss: 8.3317
Epoch 298/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.5490 - val_loss: 7.0265
Epoch 299/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.8541 - val_loss: 7.0525
Epoch 300/1000
1052/1052 [==============================] - 0s 96us/step - loss: 6.2339 - val_loss: 6.9895
Epoch 301/1000
1052/1052 [==============================] - 0s 101us/step - loss: 6.2688 - val_loss: 7.0183
Epoch 302/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.1968 - val_loss: 7.9423
Epoch 303/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.3431 - val_loss: 8.8257
Epoch 304/1000
1052/1052 [========================

...

1052/1052 [==============================] - 0s 96us/step - loss: 6.9877 - val_loss: 6.1677
Epoch 752/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.1202 - val_loss: 7.3664
Epoch 753/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.5588 - val_loss: 6.0737
Epoch 754/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.4794 - val_loss: 8.5081
Epoch 755/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.7465 - val_loss: 6.3786
Epoch 756/1000
1052/1052 [==============================] - 0s 93us/step - loss: 6.0567 - val_loss: 6.6653
Epoch 757/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.7259 - val_loss: 6.7777
Epoch 758/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.0373 - val_loss: 6.4517
Epoch 759/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.6524 - val_loss: 6.8236
Epoch 760/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 90us/step - loss: 5.9766 - val_loss: 6.6450
Epoch 828/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.6095 - val_loss: 6.0995
Epoch 829/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.1714 - val_loss: 7.0004
Epoch 830/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.5975 - val_loss: 6.8654
Epoch 831/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.6257 - val_loss: 6.4789
Epoch 832/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6186 - val_loss: 7.6932
Epoch 833/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.0805 - val_loss: 7.5172
Epoch 834/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.6099 - val_loss: 6.6237
Epoch 835/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.6757 - val_loss: 6.7443
Epoch 836/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 93us/step - loss: 5.8589 - val_loss: 9.4899
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.9774 - val_loss: 8.5380
Epoch 905/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.9057 - val_loss: 7.7072
Epoch 906/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.9237 - val_loss: 6.8944
Epoch 907/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.5608 - val_loss: 7.3023
Epoch 908/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.9670 - val_loss: 6.7992
Epoch 909/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.0381 - val_loss: 6.8549
Epoch 910/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.6937 - val_loss: 6.4930
Epoch 911/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.6872 - val_loss: 6.6081
Epoch 912/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 6.0972 - val_loss: 8.6863
Epoch 980/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.8244 - val_loss: 6.8086
Epoch 981/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.8845 - val_loss: 6.7872
Epoch 982/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.2246 - val_loss: 10.0308
Epoch 983/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.5127 - val_loss: 9.2875
Epoch 984/1000
1052/1052 [==============================] - 0s 96us/step - loss: 5.7103 - val_loss: 7.5360
Epoch 985/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.7839 - val_loss: 10.0924
Epoch 986/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.4926 - val_loss: 6.7309
Epoch 987/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.7596 - val_loss: 6.9285
Epoch 988/1000
1052/1052 [========================

[array([[-0.15917177,  4.019571  , -0.4966733 , -0.35501915, -2.5954704 ,
          0.2608817 , -4.930172  , -2.4686968 , -0.8906447 , -3.0119762 ],
        [-0.19712326, -0.02506241, -0.73848444, -0.51074064,  2.0635738 ,
          0.6605819 ,  0.34523457,  2.0927665 , -0.7253821 ,  0.02087061],
        [ 0.62129486, -0.21315332, -0.9323829 , -0.25856674,  0.01871373,
          2.0392547 , -0.0603885 , -0.07134196, -0.29165593, -0.9626695 ],
        [ 0.23559852,  0.00578611,  0.44357622, -0.6246597 ,  5.006099  ,
         -0.14556143,  0.08674883,  0.21636571,  0.37090892,  0.12875393],
        [-0.9508229 ,  1.9738365 , -0.13214016,  0.15666112, -5.14663   ,
         -0.74210525, -0.60973364, -0.15927906, -1.3166426 , -0.46851784]],
       dtype=float32),
 array([ 2.1767712 ,  5.178756  ,  3.5349798 , -3.1752148 ,  0.15495494,
         0.7801599 , -2.6150708 , -2.080281  , -0.41649815, -3.1530745 ],
       dtype=float32),
 array([[ 0.47484997,  0.83834696, -0.0726463 , -0.16832487, 

In [37]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure6_5th.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_1(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_14 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 6         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 740us/step - loss: 5

274/274 [==============================] - 0s 106us/step - loss: 6.7972 - val_loss: 9.0262
Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6601 - val_loss: 9.1263
Epoch 70/1000
274/274 [==============================] - 0s 94us/step - loss: 6.5901 - val_loss: 9.0137
Epoch 71/1000
274/274 [==============================] - 0s 105us/step - loss: 6.5910 - val_loss: 9.2202
Epoch 72/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5255 - val_loss: 9.0799
Epoch 73/1000
274/274 [==============================] - 0s 101us/step - loss: 6.5302 - val_loss: 8.8674
Epoch 74/1000
274/274 [==============================] - 0s 94us/step - loss: 6.5614 - val_loss: 8.8424
Epoch 75/1000
274/274 [==============================] - 0s 109us/step - loss: 6.4626 - val_loss: 9.1650
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5573 - val_loss: 9.0195
Epoch 77/1000
274/274 [==============================] - 0s 102us/step -

274/274 [==============================] - 0s 98us/step - loss: 5.9661 - val_loss: 7.9377
Epoch 147/1000
274/274 [==============================] - 0s 101us/step - loss: 6.0142 - val_loss: 7.9213
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 6.0088 - val_loss: 8.2575
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1194 - val_loss: 7.8335
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3031 - val_loss: 7.9709
Epoch 151/1000
274/274 [==============================] - 0s 113us/step - loss: 5.9377 - val_loss: 8.1810
Epoch 152/1000
274/274 [==============================] - 0s 120us/step - loss: 6.1485 - val_loss: 7.7845
Epoch 153/1000
274/274 [==============================] - 0s 124us/step - loss: 5.9583 - val_loss: 7.9133
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0836 - val_loss: 7.9301
Epoch 155/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 102us/step - loss: 5.8202 - val_loss: 7.7962
Epoch 224/1000
274/274 [==============================] - 0s 91us/step - loss: 5.7981 - val_loss: 7.8512
Epoch 225/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7119 - val_loss: 7.8065
Epoch 226/1000
274/274 [==============================] - 0s 97us/step - loss: 5.7172 - val_loss: 7.9598
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8700 - val_loss: 7.6560
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9072 - val_loss: 7.9910
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9610 - val_loss: 7.8058
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7043 - val_loss: 7.9990
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8544 - val_loss: 7.9290
Epoch 232/1000
274/274 [==============================] - 0s 95us/s

274/274 [==============================] - 0s 102us/step - loss: 5.7305 - val_loss: 7.8792
Epoch 301/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7339 - val_loss: 7.9981
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8406 - val_loss: 8.0055
Epoch 303/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8985 - val_loss: 8.0487
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6647 - val_loss: 8.0797
Epoch 305/1000
274/274 [==============================] - 0s 96us/step - loss: 5.9165 - val_loss: 7.9003
Epoch 306/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7585 - val_loss: 7.9099
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7898 - val_loss: 7.8538
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7659 - val_loss: 8.2309
Epoch 309/1000
274/274 [==============================] - 0s 97us/

...

274/274 [==============================] - 0s 106us/step - loss: 5.4606 - val_loss: 9.1683
Epoch 764/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3594 - val_loss: 9.2193
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3608 - val_loss: 8.8384
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2898 - val_loss: 9.0984
Epoch 767/1000
274/274 [==============================] - 0s 103us/step - loss: 5.3852 - val_loss: 9.3553
Epoch 768/1000
274/274 [==============================] - 0s 96us/step - loss: 5.3939 - val_loss: 9.2045
Epoch 769/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3597 - val_loss: 8.8539
Epoch 770/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4025 - val_loss: 9.0976
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3341 - val_loss: 9.0256
Epoch 772/1000
274/274 [==============================] - 0s 98us

274/274 [==============================] - 0s 109us/step - loss: 5.4428 - val_loss: 8.9371
Epoch 841/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3339 - val_loss: 8.6289
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3062 - val_loss: 9.4634
Epoch 843/1000
274/274 [==============================] - 0s 99us/step - loss: 5.3053 - val_loss: 8.9643
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5352 - val_loss: 9.0716
Epoch 845/1000
274/274 [==============================] - 0s 101us/step - loss: 6.0618 - val_loss: 8.6093
Epoch 846/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3864 - val_loss: 9.0364
Epoch 847/1000
274/274 [==============================] - 0s 94us/step - loss: 5.3434 - val_loss: 8.9509
Epoch 848/1000
274/274 [==============================] - 0s 103us/step - loss: 5.3045 - val_loss: 8.9926
Epoch 849/1000
274/274 [==============================] - 0s 107

274/274 [==============================] - 0s 138us/step - loss: 5.3161 - val_loss: 9.2228
Epoch 918/1000
274/274 [==============================] - 0s 160us/step - loss: 5.5051 - val_loss: 9.5675
Epoch 919/1000
274/274 [==============================] - 0s 167us/step - loss: 5.3821 - val_loss: 8.8739
Epoch 920/1000
274/274 [==============================] - 0s 149us/step - loss: 5.3184 - val_loss: 9.0506
Epoch 921/1000
274/274 [==============================] - 0s 153us/step - loss: 5.3213 - val_loss: 9.2518
Epoch 922/1000
274/274 [==============================] - 0s 175us/step - loss: 5.3145 - val_loss: 9.2490
Epoch 923/1000
274/274 [==============================] - 0s 167us/step - loss: 5.4892 - val_loss: 9.1926
Epoch 924/1000
274/274 [==============================] - 0s 189us/step - loss: 5.3992 - val_loss: 9.0727
Epoch 925/1000
274/274 [==============================] - 0s 182us/step - loss: 5.3838 - val_loss: 9.1169
Epoch 926/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 105us/step - loss: 5.3177 - val_loss: 9.4260
Epoch 995/1000
274/274 [==============================] - 0s 113us/step - loss: 5.3107 - val_loss: 8.8335
Epoch 996/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3593 - val_loss: 8.9539
Epoch 997/1000
274/274 [==============================] - 0s 104us/step - loss: 5.2546 - val_loss: 9.2255
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3059 - val_loss: 9.2692
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2713 - val_loss: 9.0341
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3712 - val_loss: 9.1282
6.5557588237827105


[array([[ 0.99356765,  0.16494359,  0.15535271,  0.18563902,  0.6761655 ],
        [-1.4501251 ,  2.4916284 , -0.12775901, -1.355335  ,  1.581255  ],
        [ 0.8945238 ,  2.2689772 , -1.2922693 ,  1.1545163 , -0.77257264],
        [-0.2587891 , -1.7165097 ,  1.7504747 ,  0.3099132 , -1.410206  ],
        [-0.58115524,  0.9048627 , -0.24619685, -0.4446224 ,  0.1418841 ],
        [ 1.1654044 ,  0.6679163 , -1.9152089 ,  0.4524394 ,  0.03609947],
        [ 0.61408573, -0.3363335 ,  0.11003856,  0.04564286, -0.30016184]],
       dtype=float32),
 array([2.1210492 , 1.069088  , 1.5970203 , 0.87655294, 2.3812053 ],
       dtype=float32),
 array([[-0.9688265 , -1.1545979 , -0.7451217 ,  0.9702589 , -0.4648472 ],
        [ 1.1182098 ,  0.15094972,  1.095042  , -0.87730986, -0.13771516],
        [ 1.1280768 ,  0.45289543,  0.9372022 , -1.0536025 ,  0.10317706],
        [ 0.69545   ,  0.92196745,  0.99080575, -1.1879811 ,  0.5716506 ],
        [-0.32867548, -1.8834138 , -1.5012529 , -0.10615854

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure1_5th.h5')

In [40]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_2(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 808us/step - loss:

274/274 [==============================] - 0s 105us/step - loss: 7.4574 - val_loss: 8.7780
Epoch 69/1000
274/274 [==============================] - 0s 101us/step - loss: 7.6668 - val_loss: 9.0483
Epoch 70/1000
274/274 [==============================] - 0s 109us/step - loss: 7.5580 - val_loss: 8.8385
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 7.3970 - val_loss: 8.8802
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 7.2857 - val_loss: 8.9404
Epoch 73/1000
274/274 [==============================] - 0s 105us/step - loss: 7.2484 - val_loss: 9.0185
Epoch 74/1000
274/274 [==============================] - 0s 109us/step - loss: 7.3022 - val_loss: 8.9732
Epoch 75/1000
274/274 [==============================] - 0s 104us/step - loss: 7.2217 - val_loss: 9.0698
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 7.2176 - val_loss: 9.0400
Epoch 77/1000
274/274 [==============================] - 0s 101us/step

Epoch 146/1000
274/274 [==============================] - 0s 109us/step - loss: 6.8184 - val_loss: 8.7023
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8755 - val_loss: 8.4839
Epoch 148/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6771 - val_loss: 8.5054
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7147 - val_loss: 8.4862
Epoch 150/1000
274/274 [==============================] - 0s 100us/step - loss: 6.6435 - val_loss: 8.5697
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8146 - val_loss: 8.5735
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7933 - val_loss: 8.4516
Epoch 153/1000
274/274 [==============================] - 0s 101us/step - loss: 6.5695 - val_loss: 8.5854
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0095 - val_loss: 8.7341
Epoch 155/1000
274/274 [=========================

274/274 [==============================] - 0s 102us/step - loss: 6.1604 - val_loss: 8.8322
Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3321 - val_loss: 8.8223
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2287 - val_loss: 8.6929
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2317 - val_loss: 8.8848
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2603 - val_loss: 8.7368
Epoch 228/1000
274/274 [==============================] - 0s 105us/step - loss: 6.1145 - val_loss: 8.8224
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2126 - val_loss: 8.7373
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6225 - val_loss: 8.7365
Epoch 231/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0124 - val_loss: 8.7921
Epoch 232/1000
274/274 [==============================] - 0s 98

274/274 [==============================] - 0s 102us/step - loss: 5.8703 - val_loss: 8.8910
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9306 - val_loss: 8.8945
Epoch 302/1000
274/274 [==============================] - 0s 104us/step - loss: 5.9988 - val_loss: 8.7040
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9562 - val_loss: 8.7329
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9171 - val_loss: 9.0125
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8362 - val_loss: 8.8297
Epoch 306/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8492 - val_loss: 8.9064
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9064 - val_loss: 9.0509
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8655 - val_loss: 8.8953
Epoch 309/1000
274/274 [==============================] - 0s 102

...

274/274 [==============================] - 0s 109us/step - loss: 5.4872 - val_loss: 9.2205
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4755 - val_loss: 9.0641
Epoch 764/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4843 - val_loss: 9.0578
Epoch 765/1000
274/274 [==============================] - 0s 103us/step - loss: 5.4604 - val_loss: 9.1502
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4437 - val_loss: 9.1433
Epoch 767/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4489 - val_loss: 9.1882
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4715 - val_loss: 9.1027
Epoch 769/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4683 - val_loss: 9.5024
Epoch 770/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4201 - val_loss: 9.3643
Epoch 771/1000
274/274 [==============================] - 0s 98

274/274 [==============================] - 0s 106us/step - loss: 5.4948 - val_loss: 9.3060
Epoch 840/1000
274/274 [==============================] - 0s 101us/step - loss: 5.3754 - val_loss: 9.3381
Epoch 841/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4095 - val_loss: 9.1666
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5784 - val_loss: 9.1558
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4648 - val_loss: 9.3291
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3971 - val_loss: 9.4610
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4234 - val_loss: 9.4679
Epoch 846/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3721 - val_loss: 9.4503
Epoch 847/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4405 - val_loss: 9.5957
Epoch 848/1000
274/274 [==============================] - 0s 102

274/274 [==============================] - 0s 116us/step - loss: 5.4592 - val_loss: 9.8373
Epoch 917/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4951 - val_loss: 9.7126
Epoch 918/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9120 - val_loss: 9.3989
Epoch 919/1000
274/274 [==============================] - 0s 103us/step - loss: 5.9431 - val_loss: 9.4595
Epoch 920/1000
274/274 [==============================] - 0s 104us/step - loss: 6.3404 - val_loss: 9.6095
Epoch 921/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7855 - val_loss: 9.4991
Epoch 922/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4124 - val_loss: 9.5290
Epoch 923/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3509 - val_loss: 9.6673
Epoch 924/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4475 - val_loss: 9.4999
Epoch 925/1000
274/274 [==============================] - 0s 98

274/274 [==============================] - 0s 102us/step - loss: 5.8566 - val_loss: 9.8788
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4995 - val_loss: 9.5433
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3729 - val_loss: 9.4641
Epoch 996/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3871 - val_loss: 9.6308
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3773 - val_loss: 9.5392
Epoch 998/1000
274/274 [==============================] - 0s 113us/step - loss: 5.4859 - val_loss: 9.3649
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3817 - val_loss: 9.3953
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4783 - val_loss: 9.4770
9.071677515062236


[array([[-0.51874214,  3.8535974 ,  2.2751915 , -3.021215  , -0.7872916 ],
        [-0.03733763, -2.429842  ,  1.5366368 , -0.89319515, -0.36674565],
        [ 0.72795284, -0.3823999 , -0.28429425,  0.56650466,  0.16354705],
        [ 0.35076463,  3.2556636 , -1.7930532 ,  0.9477738 ,  0.91726685],
        [-0.7418415 , -0.17462517,  0.4575346 ,  0.18158932,  0.6567774 ],
        [ 0.5216368 , -1.0902793 , -3.924542  , -1.4403158 ,  0.12760918],
        [ 0.30621868, -0.32432774, -2.727077  ,  0.05527413, -1.8019938 ]],
       dtype=float32),
 array([ 0.45845428,  0.7437406 , -3.5960498 , -1.6679076 , -1.8046108 ],
       dtype=float32),
 array([[-0.9705552 ,  0.02390792,  0.39458612, -0.19922191, -0.627256  ,
          0.7761993 , -0.59901744,  0.26018935,  0.94620734,  0.16193284],
        [-0.3873798 ,  1.4240611 , -0.36167154,  0.83230406, -1.1943715 ,
          0.30226052, -0.6800488 ,  0.39711514,  0.8206726 ,  0.86138636],
        [-0.86163485,  0.6835003 ,  0.42114133, -0.25590

In [41]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure2_5th.h5')

In [42]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_3(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_18 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 16        
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 866us/step - loss:

274/274 [==============================] - 0s 102us/step - loss: 7.1550 - val_loss: 10.5397
Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 7.2434 - val_loss: 10.6209
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0862 - val_loss: 10.4551
Epoch 71/1000
274/274 [==============================] - 0s 104us/step - loss: 7.1638 - val_loss: 10.6076
Epoch 72/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0960 - val_loss: 10.8479
Epoch 73/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0929 - val_loss: 10.5361
Epoch 74/1000
274/274 [==============================] - 0s 100us/step - loss: 7.1474 - val_loss: 10.5274
Epoch 75/1000
274/274 [==============================] - 0s 102us/step - loss: 7.2400 - val_loss: 10.4862
Epoch 76/1000
274/274 [==============================] - 0s 104us/step - loss: 7.3418 - val_loss: 10.6198
Epoch 77/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 106us/step - loss: 6.9951 - val_loss: 10.5718
Epoch 146/1000
274/274 [==============================] - 0s 113us/step - loss: 6.6885 - val_loss: 10.2163
Epoch 147/1000
274/274 [==============================] - 0s 100us/step - loss: 6.9848 - val_loss: 9.9192
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8554 - val_loss: 10.1662
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8976 - val_loss: 10.3437
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8114 - val_loss: 10.1009
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9980 - val_loss: 10.2027
Epoch 152/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0703 - val_loss: 10.4081
Epoch 153/1000
274/274 [==============================] - 0s 104us/step - loss: 6.9334 - val_loss: 10.3981
Epoch 154/1000
274/274 [==============================

Epoch 222/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5857 - val_loss: 10.5778
Epoch 223/1000
274/274 [==============================] - 0s 105us/step - loss: 6.6163 - val_loss: 10.5271
Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4110 - val_loss: 10.3229
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6768 - val_loss: 10.2312
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6259 - val_loss: 10.4473
Epoch 227/1000
274/274 [==============================] - 0s 105us/step - loss: 6.4179 - val_loss: 10.3329
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4982 - val_loss: 10.5446
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3923 - val_loss: 10.2992
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3428 - val_loss: 10.4352
Epoch 231/1000
274/274 [===============

Epoch 299/1000
274/274 [==============================] - 0s 105us/step - loss: 6.0989 - val_loss: 10.6600
Epoch 300/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0224 - val_loss: 10.6070
Epoch 301/1000
274/274 [==============================] - 0s 106us/step - loss: 6.0425 - val_loss: 10.7331
Epoch 302/1000
274/274 [==============================] - 0s 101us/step - loss: 6.0014 - val_loss: 10.4868
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8269 - val_loss: 10.4385
Epoch 304/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8879 - val_loss: 9.9182
Epoch 305/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2364 - val_loss: 10.8245
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2154 - val_loss: 10.6327
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9610 - val_loss: 10.4052
Epoch 308/1000
274/274 [================

...

Epoch 761/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3149 - val_loss: 10.3283
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5837 - val_loss: 10.4213
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4685 - val_loss: 10.5751
Epoch 764/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4592 - val_loss: 10.2879
Epoch 765/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6201 - val_loss: 10.2664
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3736 - val_loss: 10.5161
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2951 - val_loss: 10.4458
Epoch 768/1000
274/274 [==============================] - 0s 109us/step - loss: 5.2519 - val_loss: 10.4169
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3442 - val_loss: 10.7246
Epoch 770/1000
274/274 [================

274/274 [==============================] - 0s 106us/step - loss: 5.8030 - val_loss: 10.9490
Epoch 838/1000
274/274 [==============================] - 0s 107us/step - loss: 5.4402 - val_loss: 11.2336
Epoch 839/1000
274/274 [==============================] - 0s 97us/step - loss: 5.5232 - val_loss: 11.1528
Epoch 840/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6445 - val_loss: 10.5036
Epoch 841/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3333 - val_loss: 10.9452
Epoch 842/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4199 - val_loss: 11.0184
Epoch 843/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3532 - val_loss: 11.1492
Epoch 844/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2736 - val_loss: 10.5540
Epoch 845/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3990 - val_loss: 10.5856
Epoch 846/1000
274/274 [==============================

Epoch 914/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2505 - val_loss: 10.7106
Epoch 915/1000
274/274 [==============================] - 0s 103us/step - loss: 5.4890 - val_loss: 11.1053
Epoch 916/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3112 - val_loss: 11.1822
Epoch 917/1000
274/274 [==============================] - 0s 103us/step - loss: 5.2662 - val_loss: 11.4369
Epoch 918/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2507 - val_loss: 11.3143
Epoch 919/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3338 - val_loss: 10.9135
Epoch 920/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4414 - val_loss: 11.2803
Epoch 921/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2097 - val_loss: 11.0558
Epoch 922/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2130 - val_loss: 11.2568
Epoch 923/1000
274/274 [==============

274/274 [==============================] - 0s 102us/step - loss: 5.5971 - val_loss: 11.9392
Epoch 991/1000
274/274 [==============================] - 0s 99us/step - loss: 5.6393 - val_loss: 11.5101
Epoch 992/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7065 - val_loss: 11.5139
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7981 - val_loss: 11.2188
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2631 - val_loss: 11.3841
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2806 - val_loss: 11.4312
Epoch 996/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2910 - val_loss: 11.3550
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1255 - val_loss: 11.2765
Epoch 998/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4288 - val_loss: 11.2869
Epoch 999/1000
274/274 [==============================] 

[array([[ 5.9947214 ,  2.1603937 ,  2.092438  , -1.7962257 ,  0.72769296],
        [-4.15684   , -0.27200875, -2.4469073 ,  0.93143123, -0.10899017],
        [ 1.9528196 , -0.5212081 ,  0.61850023, -1.4901567 , -0.40282857],
        [ 2.7759283 , -0.08122619,  2.2482688 ,  1.2712804 , -0.27706063],
        [ 0.47253838, -0.1181616 , -1.5084828 , -1.076795  , -0.86041474],
        [-0.93567103,  0.43867385, -0.7143239 ,  0.450369  ,  0.21587536],
        [-1.4327155 ,  0.1297445 , -0.38170317,  0.79186046,  2.6668472 ]],
       dtype=float32),
 array([ 1.7499155 ,  0.83572453, -2.5078278 , -0.88122225,  2.0698762 ],
       dtype=float32),
 array([[-1.3948843 ,  0.4887941 ,  1.4077668 , -1.358068  ,  0.7383147 ,
          0.60628116,  0.5940842 , -1.0893748 ,  1.2588139 , -0.15023941,
          0.99476963, -1.3742934 , -0.8014059 , -0.30399513,  0.16127667],
        [ 1.1718462 , -1.056107  , -1.1746718 ,  1.0774827 , -0.59184355,
         -0.85332197, -0.35659194,  0.7208892 , -1.479712

In [43]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure3_5th.h5')

In [44]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_4(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_19 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_20 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 987us/step - loss

274/274 [==============================] - 0s 98us/step - loss: 6.3382 - val_loss: 8.6799
Epoch 69/1000
274/274 [==============================] - 0s 113us/step - loss: 6.2357 - val_loss: 8.5183
Epoch 70/1000
274/274 [==============================] - 0s 103us/step - loss: 6.1543 - val_loss: 8.4019
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2414 - val_loss: 8.6572
Epoch 72/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5672 - val_loss: 8.8749
Epoch 73/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2446 - val_loss: 8.8955
Epoch 74/1000
274/274 [==============================] - 0s 99us/step - loss: 6.1644 - val_loss: 8.7319
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2062 - val_loss: 8.1324
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3106 - val_loss: 8.2925
Epoch 77/1000
274/274 [==============================] - 0s 105us/step - 

Epoch 146/1000
274/274 [==============================] - 0s 103us/step - loss: 5.4266 - val_loss: 8.5885
Epoch 147/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4353 - val_loss: 8.3063
Epoch 148/1000
274/274 [==============================] - 0s 110us/step - loss: 5.6293 - val_loss: 8.9246
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5730 - val_loss: 9.3827
Epoch 150/1000
274/274 [==============================] - 0s 107us/step - loss: 5.6520 - val_loss: 8.6697
Epoch 151/1000
274/274 [==============================] - 0s 101us/step - loss: 5.3492 - val_loss: 8.9056
Epoch 152/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4222 - val_loss: 8.6946
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5235 - val_loss: 9.0524
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3422 - val_loss: 9.0114
Epoch 155/1000
274/274 [========================

274/274 [==============================] - 0s 102us/step - loss: 4.9829 - val_loss: 8.7864
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0228 - val_loss: 9.1049
Epoch 225/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1322 - val_loss: 8.9393
Epoch 226/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0788 - val_loss: 9.2063
Epoch 227/1000
274/274 [==============================] - 0s 109us/step - loss: 5.0616 - val_loss: 8.8921
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0110 - val_loss: 8.8232
Epoch 229/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0983 - val_loss: 8.7058
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1783 - val_loss: 9.0473
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2809 - val_loss: 8.9344
Epoch 232/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 105us/step - loss: 4.9565 - val_loss: 9.4080
Epoch 301/1000
274/274 [==============================] - 0s 109us/step - loss: 4.9179 - val_loss: 9.0772
Epoch 302/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7902 - val_loss: 9.2181
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7427 - val_loss: 9.0220
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8160 - val_loss: 9.0257
Epoch 305/1000
274/274 [==============================] - 0s 104us/step - loss: 4.7418 - val_loss: 9.0126
Epoch 306/1000
274/274 [==============================] - 0s 99us/step - loss: 4.7636 - val_loss: 8.9607
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7473 - val_loss: 9.1022
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8710 - val_loss: 9.0014
Epoch 309/1000
274/274 [==============================] - 0s 98u

...

274/274 [==============================] - 0s 98us/step - loss: 3.6351 - val_loss: 8.4861
Epoch 763/1000
274/274 [==============================] - 0s 109us/step - loss: 3.8334 - val_loss: 7.8587
Epoch 764/1000
274/274 [==============================] - 0s 102us/step - loss: 3.6654 - val_loss: 8.0386
Epoch 765/1000
274/274 [==============================] - 0s 103us/step - loss: 3.7636 - val_loss: 7.6043
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7641 - val_loss: 7.8867
Epoch 767/1000
274/274 [==============================] - 0s 109us/step - loss: 3.6940 - val_loss: 8.1977
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7373 - val_loss: 7.8881
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7248 - val_loss: 8.2769
Epoch 770/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0499 - val_loss: 7.7166
Epoch 771/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 102us/step - loss: 3.7077 - val_loss: 8.2757
Epoch 840/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7199 - val_loss: 7.8420
Epoch 841/1000
274/274 [==============================] - 0s 111us/step - loss: 4.3486 - val_loss: 10.2480
Epoch 842/1000
274/274 [==============================] - 0s 100us/step - loss: 4.0932 - val_loss: 8.1153
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7766 - val_loss: 8.4650
Epoch 844/1000
274/274 [==============================] - 0s 96us/step - loss: 3.5358 - val_loss: 7.6364
Epoch 845/1000
274/274 [==============================] - 0s 113us/step - loss: 3.9384 - val_loss: 7.8675
Epoch 846/1000
274/274 [==============================] - 0s 106us/step - loss: 3.5641 - val_loss: 7.8853
Epoch 847/1000
274/274 [==============================] - 0s 101us/step - loss: 3.5417 - val_loss: 7.6711
Epoch 848/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 120us/step - loss: 4.1595 - val_loss: 8.9834
Epoch 917/1000
274/274 [==============================] - 0s 109us/step - loss: 3.7806 - val_loss: 7.8353
Epoch 918/1000
274/274 [==============================] - 0s 101us/step - loss: 3.6358 - val_loss: 8.3170
Epoch 919/1000
274/274 [==============================] - 0s 101us/step - loss: 3.4606 - val_loss: 7.7311
Epoch 920/1000
274/274 [==============================] - 0s 102us/step - loss: 3.5185 - val_loss: 7.7413
Epoch 921/1000
274/274 [==============================] - 0s 106us/step - loss: 3.4835 - val_loss: 7.9551
Epoch 922/1000
274/274 [==============================] - 0s 98us/step - loss: 3.5365 - val_loss: 7.7658
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6893 - val_loss: 8.4265
Epoch 924/1000
274/274 [==============================] - 0s 106us/step - loss: 3.6208 - val_loss: 7.8458
Epoch 925/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 102us/step - loss: 3.5395 - val_loss: 7.8374
Epoch 994/1000
274/274 [==============================] - 0s 105us/step - loss: 3.8849 - val_loss: 9.2859
Epoch 995/1000
274/274 [==============================] - 0s 106us/step - loss: 3.7206 - val_loss: 7.8367
Epoch 996/1000
274/274 [==============================] - 0s 109us/step - loss: 3.5748 - val_loss: 8.7890
Epoch 997/1000
274/274 [==============================] - 0s 101us/step - loss: 3.7292 - val_loss: 8.2017
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 3.4603 - val_loss: 8.1120
Epoch 999/1000
274/274 [==============================] - 0s 106us/step - loss: 3.6156 - val_loss: 7.5798
Epoch 1000/1000
274/274 [==============================] - 0s 106us/step - loss: 3.6022 - val_loss: 7.9730
8.809314226700089


[array([[-8.7864208e+00,  7.6267326e-01, -2.4002421e+00, -1.7280487e+00,
         -4.1630650e+00, -2.3844576e+00, -5.3514713e-01,  1.4467013e+00,
         -1.4786328e+00, -2.9776237e+00],
        [ 4.3011904e+00,  5.6811059e-01,  7.5648427e-01, -1.9832537e+00,
          4.0114980e+00, -2.0964696e+00, -5.6311917e-01,  7.3642260e-01,
         -2.1150808e-01, -1.5615532e+00],
        [-1.0612311e+00, -1.2349058e+00, -3.0803852e-02, -2.9918370e+00,
         -3.7360594e-01, -6.5256411e-01,  4.5209745e-01, -1.3624247e+00,
          1.3914407e+00,  7.4851239e-01],
        [-2.6732850e+00,  4.6401474e-01,  3.0440798e-01, -8.2014389e-02,
         -3.4087372e+00,  2.6543715e+00,  1.0478257e+00,  1.5046620e-01,
         -3.5844037e-01,  5.1586467e-01],
        [ 1.7730545e-01,  4.6385849e-01, -7.8923684e-03, -1.8282443e+00,
          2.0283082e+00, -9.8568851e-01,  5.3708541e-01,  1.0762062e+00,
         -1.1287360e+00,  9.3779892e-02],
        [ 1.1659927e+00, -9.3233806e-01,  7.7578716e-02, -1.

In [45]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure4_5th.h5')

In [46]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_5(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_21 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 992us/step - loss

274/274 [==============================] - 0s 109us/step - loss: 6.6191 - val_loss: 10.2011
Epoch 69/1000
274/274 [==============================] - 0s 106us/step - loss: 6.6932 - val_loss: 11.0334
Epoch 70/1000
274/274 [==============================] - 0s 99us/step - loss: 6.9525 - val_loss: 10.2748
Epoch 71/1000
274/274 [==============================] - 0s 104us/step - loss: 6.5840 - val_loss: 10.9012
Epoch 72/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4958 - val_loss: 10.2440
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5515 - val_loss: 10.2471
Epoch 74/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6199 - val_loss: 11.0670
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4899 - val_loss: 10.2531
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4075 - val_loss: 10.6322
Epoch 77/1000
274/274 [==============================] - 0s 111u

274/274 [==============================] - 0s 102us/step - loss: 5.7444 - val_loss: 9.9291
Epoch 146/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7589 - val_loss: 10.7733
Epoch 147/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1687 - val_loss: 10.3883
Epoch 148/1000
274/274 [==============================] - 0s 100us/step - loss: 6.1397 - val_loss: 11.1819
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9701 - val_loss: 10.2631
Epoch 150/1000
274/274 [==============================] - 0s 113us/step - loss: 5.8237 - val_loss: 10.9290
Epoch 151/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7189 - val_loss: 9.7691
Epoch 152/1000
274/274 [==============================] - 0s 99us/step - loss: 5.7803 - val_loss: 10.6120
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7355 - val_loss: 9.5711
Epoch 154/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 113us/step - loss: 5.3109 - val_loss: 9.0126
Epoch 223/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7425 - val_loss: 9.7125
Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2729 - val_loss: 8.9892
Epoch 225/1000
274/274 [==============================] - 0s 101us/step - loss: 5.1560 - val_loss: 10.0639
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1403 - val_loss: 9.0515
Epoch 227/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1815 - val_loss: 8.8849
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0537 - val_loss: 9.0099
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0834 - val_loss: 9.4112
Epoch 230/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1936 - val_loss: 8.9900
Epoch 231/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 106us/step - loss: 5.2834 - val_loss: 8.7682
Epoch 300/1000
274/274 [==============================] - 0s 111us/step - loss: 4.9482 - val_loss: 9.7013
Epoch 301/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4152 - val_loss: 8.5380
Epoch 302/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9793 - val_loss: 8.9231
Epoch 303/1000
274/274 [==============================] - 0s 101us/step - loss: 4.7682 - val_loss: 8.5826
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8430 - val_loss: 9.1954
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8073 - val_loss: 9.2046
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9077 - val_loss: 8.8938
Epoch 307/1000
274/274 [==============================] - 0s 99us/step - loss: 4.8737 - val_loss: 8.7904
Epoch 308/1000
274/274 [==============================] - 0s 1

...

274/274 [==============================] - 0s 102us/step - loss: 4.0769 - val_loss: 8.5512
Epoch 762/1000
274/274 [==============================] - 0s 104us/step - loss: 4.3630 - val_loss: 8.7006
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2831 - val_loss: 8.4053
Epoch 764/1000
274/274 [==============================] - 0s 105us/step - loss: 4.1707 - val_loss: 8.6522
Epoch 765/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0785 - val_loss: 8.6223
Epoch 766/1000
274/274 [==============================] - 0s 101us/step - loss: 4.2404 - val_loss: 8.6193
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0584 - val_loss: 8.7891
Epoch 768/1000
274/274 [==============================] - 0s 100us/step - loss: 4.0586 - val_loss: 8.2860
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0110 - val_loss: 8.2491
Epoch 770/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 102us/step - loss: 4.0004 - val_loss: 8.4085
Epoch 839/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1361 - val_loss: 8.7605
Epoch 840/1000
274/274 [==============================] - 0s 105us/step - loss: 4.0413 - val_loss: 8.2641
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1292 - val_loss: 8.7545
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9798 - val_loss: 8.5007
Epoch 843/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0401 - val_loss: 8.6662
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0684 - val_loss: 8.3399
Epoch 845/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0564 - val_loss: 8.5915
Epoch 846/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1383 - val_loss: 8.7192
Epoch 847/1000
274/274 [==============================] - 0s 106

274/274 [==============================] - 0s 98us/step - loss: 4.0168 - val_loss: 8.6883
Epoch 916/1000
274/274 [==============================] - 0s 109us/step - loss: 4.0279 - val_loss: 8.4772
Epoch 917/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9949 - val_loss: 9.0520
Epoch 918/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1008 - val_loss: 8.8853
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1514 - val_loss: 8.4090
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0454 - val_loss: 8.7317
Epoch 921/1000
274/274 [==============================] - 0s 97us/step - loss: 4.0409 - val_loss: 8.8228
Epoch 922/1000
274/274 [==============================] - 0s 102us/step - loss: 4.4459 - val_loss: 9.0959
Epoch 923/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2503 - val_loss: 10.1916
Epoch 924/1000
274/274 [==============================] - 0s 98us

274/274 [==============================] - 0s 95us/step - loss: 4.0270 - val_loss: 9.1207
Epoch 993/1000
274/274 [==============================] - 0s 105us/step - loss: 4.0752 - val_loss: 9.2064
Epoch 994/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0849 - val_loss: 8.9553
Epoch 995/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9762 - val_loss: 8.9602
Epoch 996/1000
274/274 [==============================] - 0s 101us/step - loss: 3.9351 - val_loss: 8.6470
Epoch 997/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2680 - val_loss: 8.7133
Epoch 998/1000
274/274 [==============================] - 0s 101us/step - loss: 3.9426 - val_loss: 9.3683
Epoch 999/1000
274/274 [==============================] - 0s 100us/step - loss: 4.0791 - val_loss: 9.0661
Epoch 1000/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1000 - val_loss: 8.6873
12.727670637227721


[array([[ 1.1313021 , -3.085816  ,  1.927065  , -2.7717645 , -0.7528484 ,
          3.0921245 , -0.56679404, -2.6540577 , -1.4992825 , -0.10916438],
        [ 0.17657736,  2.7950933 , -2.3319662 , -1.6171544 ,  1.7938892 ,
         -0.83670485, -1.9791273 , -0.38262868, -0.60952485,  0.6687368 ],
        [-1.556922  , -0.61373425,  0.02359732, -0.6884323 , -0.25900364,
         -1.6079075 ,  0.09938377,  0.85870457, -0.9627764 ,  0.811659  ],
        [ 0.33005598, -2.5487976 ,  1.437822  ,  1.2607117 , -1.1283998 ,
          0.81424636,  0.8093724 ,  0.53105897,  2.1964357 , -1.4426455 ],
        [-0.49514183, -0.17535456, -0.8312414 , -0.30191547, -0.08503196,
          1.3110816 ,  0.11915448, -0.6774414 , -0.47471035,  0.5783829 ],
        [-0.32195765, -0.3910366 ,  0.57404995, -0.8167367 , -1.783452  ,
         -1.8777193 ,  1.7414455 ,  0.61654943, -0.5295694 ,  1.7261318 ],
        [ 0.12503324,  1.2568306 ,  0.77544165, -0.19657214,  1.3289266 ,
         -3.5433874 ,  0.8947283

In [47]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure5_5th.h5')

In [48]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_6(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_23 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_24 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 16        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

274/274 [==============================] - 0s 124us/step - loss: 6.9386 - val_loss: 9.3848
Epoch 69/1000
274/274 [==============================] - ETA: 0s - loss: 3.871 - 0s 109us/step - loss: 7.6235 - val_loss: 10.1675
Epoch 70/1000
274/274 [==============================] - 0s 109us/step - loss: 7.0960 - val_loss: 9.7996
Epoch 71/1000
274/274 [==============================] - 0s 120us/step - loss: 6.6779 - val_loss: 9.5047
Epoch 72/1000
274/274 [==============================] - 0s 109us/step - loss: 6.7044 - val_loss: 9.3571
Epoch 73/1000
274/274 [==============================] - 0s 106us/step - loss: 6.6742 - val_loss: 9.4451
Epoch 74/1000
274/274 [==============================] - 0s 113us/step - loss: 6.6776 - val_loss: 9.5682
Epoch 75/1000
274/274 [==============================] - 0s 113us/step - loss: 6.6381 - val_loss: 9.4929
Epoch 76/1000
274/274 [==============================] - 0s 106us/step - loss: 6.6909 - val_loss: 9.5134
Epoch 77/1000
274/274 [=====================

274/274 [==============================] - 0s 98us/step - loss: 5.9036 - val_loss: 9.3776
Epoch 146/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1309 - val_loss: 9.0986
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1890 - val_loss: 9.1293
Epoch 148/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7989 - val_loss: 9.3588
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9046 - val_loss: 10.0181
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9498 - val_loss: 9.2121
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2748 - val_loss: 9.8890
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1569 - val_loss: 8.9019
Epoch 153/1000
274/274 [==============================] - 0s 97us/step - loss: 5.9537 - val_loss: 9.2107
Epoch 154/1000
274/274 [==============================] - 0s 94us

274/274 [==============================] - 0s 101us/step - loss: 5.0828 - val_loss: 9.6009
Epoch 223/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1521 - val_loss: 10.0698
Epoch 224/1000
274/274 [==============================] - 0s 99us/step - loss: 5.3431 - val_loss: 10.1354
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9425 - val_loss: 9.9751
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1052 - val_loss: 10.1111
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1533 - val_loss: 10.2298
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4684 - val_loss: 11.7675
Epoch 229/1000
274/274 [==============================] - 0s 101us/step - loss: 5.6756 - val_loss: 9.9090
Epoch 230/1000
274/274 [==============================] - 0s 99us/step - loss: 5.3525 - val_loss: 10.5427
Epoch 231/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 106us/step - loss: 5.3795 - val_loss: 10.8064
Epoch 300/1000
274/274 [==============================] - 0s 109us/step - loss: 4.8591 - val_loss: 9.9946
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7417 - val_loss: 10.8429
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 4.8638 - val_loss: 11.2246
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7963 - val_loss: 10.9854
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7833 - val_loss: 11.1279
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7369 - val_loss: 11.0101
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5638 - val_loss: 10.4582
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7009 - val_loss: 11.3042
Epoch 308/1000
274/274 [==============================] 

...

274/274 [==============================] - 0s 102us/step - loss: 4.0205 - val_loss: 12.2734
Epoch 760/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1312 - val_loss: 12.6683
Epoch 761/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9862 - val_loss: 12.8910
Epoch 762/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3068 - val_loss: 13.3273
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0396 - val_loss: 13.1687
Epoch 764/1000
274/274 [==============================] - 0s 106us/step - loss: 3.8481 - val_loss: 12.3985
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8513 - val_loss: 12.3536
Epoch 766/1000
274/274 [==============================] - 0s 106us/step - loss: 3.8746 - val_loss: 12.8815
Epoch 767/1000
274/274 [==============================] - 0s 99us/step - loss: 4.1060 - val_loss: 12.6469
Epoch 768/1000
274/274 [==============================]

Epoch 836/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8167 - val_loss: 12.7485
Epoch 837/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9686 - val_loss: 12.3360
Epoch 838/1000
274/274 [==============================] - 0s 95us/step - loss: 4.3841 - val_loss: 13.0537
Epoch 839/1000
274/274 [==============================] - 0s 105us/step - loss: 4.0359 - val_loss: 13.3552
Epoch 840/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9516 - val_loss: 12.8598
Epoch 841/1000
274/274 [==============================] - 0s 100us/step - loss: 3.8334 - val_loss: 12.9180
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9156 - val_loss: 12.9501
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8779 - val_loss: 13.1575
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8441 - val_loss: 12.2963
Epoch 845/1000
274/274 [===============

Epoch 913/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8656 - val_loss: 13.2620
Epoch 914/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9451 - val_loss: 14.4127
Epoch 915/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6447 - val_loss: 12.5520
Epoch 916/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9814 - val_loss: 12.7219
Epoch 917/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0734 - val_loss: 13.4563
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2206 - val_loss: 13.5664
Epoch 919/1000
274/274 [==============================] - 0s 98us/step - loss: 4.4420 - val_loss: 13.4480
Epoch 920/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3028 - val_loss: 13.1369
Epoch 921/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1987 - val_loss: 12.7831
Epoch 922/1000
274/274 [==================

Epoch 990/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7630 - val_loss: 12.9430
Epoch 991/1000
274/274 [==============================] - 0s 95us/step - loss: 3.6669 - val_loss: 13.0172
Epoch 992/1000
274/274 [==============================] - 0s 102us/step - loss: 3.6877 - val_loss: 12.8849
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8401 - val_loss: 12.9996
Epoch 994/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9854 - val_loss: 13.4278
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 3.6766 - val_loss: 12.7849
Epoch 996/1000
274/274 [==============================] - 0s 106us/step - loss: 3.6294 - val_loss: 12.5243
Epoch 997/1000
274/274 [==============================] - 0s 97us/step - loss: 3.6603 - val_loss: 13.2259
Epoch 998/1000
274/274 [==============================] - 0s 91us/step - loss: 3.8571 - val_loss: 12.5389
Epoch 999/1000
274/274 [==================

[array([[-1.74368   ,  1.7133001 ,  0.25153714,  2.4530897 , -0.60649467,
         -2.7800236 ,  0.17223884, -2.328713  , -4.0844045 , -1.2376384 ],
        [ 1.0465827 , -1.7843602 ,  1.4337764 ,  4.73622   , -0.36702675,
         -1.1285523 ,  0.33607495, -0.06482416,  2.180313  , -0.21038881],
        [ 0.03352092,  0.30336407, -0.34950233,  6.152499  ,  0.50855255,
          0.92281836, -0.5262648 , -0.00723746, -0.36696988, -0.12285184],
        [-0.18452948,  1.9864478 , -1.5635035 , -1.8603466 ,  1.039308  ,
          0.15082143, -0.9892345 ,  0.79273665, -1.713075  ,  0.9506096 ],
        [ 0.09883347, -1.344607  ,  0.18596324,  0.49432337,  1.0163242 ,
          0.04321545,  0.0231383 , -0.40512738,  0.25061497, -0.2389735 ],
        [ 0.15451103,  0.04828291,  1.5626211 , -0.742409  ,  0.5056832 ,
         -1.8813436 ,  0.41858533, -0.05174155,  0.7161473 ,  2.3426726 ],
        [ 0.06106795, -0.3065995 ,  1.3134176 ,  0.16400507, -2.64701   ,
         -0.01288791, -0.0753181

In [49]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure6_5th.h5')

In [50]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_1(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_26 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - ETA: 0s - loss: 0.004 - 0s 119us/step - loss: 0.0058 - val_loss: 0.0049
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0063 - val_loss: 0.0032
Epoch 71/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0055 - val_loss: 0.0039
Epoch 72/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0050 - val_loss: 0.0034
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0048
Epoch 74/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0055 - val_loss: 0.0041
Epoch 75/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0054 - val_loss: 0.0038
Epoch 76/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0055 - val_loss: 0.0038
Epoch 77/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0047 - val_loss: 0.0036
Epoch 78/1000
143/143 [======================

143/143 [==============================] - 0s 111us/step - loss: 0.0032 - val_loss: 0.0039
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0036
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0034 - val_loss: 0.0035
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0028
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0027
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0040
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0036
Epoch 153/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0040 - val_loss: 0.0033
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0033
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0031 - val_loss: 0.0033
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0028
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0026
Epoch 226/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0024
Epoch 227/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0027 - val_loss: 0.0028
Epoch 228/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0025 - val_loss: 0.0024
Epoch 229/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0025 - val_loss: 0.0026
Epoch 230/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0024 - val_loss: 0.0024
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0024 - val_loss: 0.0024
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0026
Epoch 301/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0025
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0026
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0025
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0023
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0027
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0031
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0026
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0038
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0039
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0048
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0043
Epoch 769/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0038
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0042
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0043
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0043
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 9.5905e-04 - val_loss: 0.0043
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0010 - val_loss: 0.0044
Epoch 918/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0012 - val_loss: 0.0051
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 9.4790e-04 - val_loss: 0.0043
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0045
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0045
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0012 - val_loss: 0.0050
Epoch 925/1000
143/143 [=============================

Epoch 993/1000
143/143 [==============================] - 0s 119us/step - loss: 9.5590e-04 - val_loss: 0.0049
Epoch 994/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0047
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0048
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0047
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0052
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0044
Epoch 999/1000
143/143 [==============================] - 0s 119us/step - loss: 9.0760e-04 - val_loss: 0.0056
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0045
0.005807793699204922


[array([[ 7.90543318e-01,  2.32354224e-01,  1.73854142e-01,
         -2.78059579e-02, -9.76069033e-01],
        [ 2.95857191e-01,  1.90213144e-01, -5.27419269e-01,
         -5.06315827e-01,  1.34484458e+00],
        [-6.74630523e-01, -1.68378785e-01, -3.54216546e-01,
         -8.35229158e-02,  6.96405351e-01],
        [ 1.62511981e+00,  3.28252614e-01,  8.35627094e-02,
          1.88651234e-01,  1.71503916e-01],
        [ 2.46903226e-01,  2.08581388e-01,  6.49674356e-01,
         -6.07086420e-01, -1.95436239e-01],
        [ 8.01487923e-01,  5.34134448e-01, -1.93868652e-01,
         -4.12134290e-01, -1.71646520e-01],
        [-6.40931487e-01,  5.79086132e-03, -1.09814811e+00,
         -1.51550606e-01,  5.69350958e-01],
        [-1.13744754e-04, -1.06804550e-01, -5.24082661e-01,
          9.20144916e-01,  7.10977390e-02],
        [-5.00250041e-01,  3.56020182e-01,  2.50352803e-03,
         -2.81417519e-01, -1.06464517e+00],
        [-3.13393176e-01, -1.09153891e+00, -8.71781588e-01,
    

In [51]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure1_5th.h5')

In [52]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_2(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 139us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 70/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0052 - val_loss: 0.0039
Epoch 71/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0048 - val_loss: 0.0041
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0049 - val_loss: 0.0040
Epoch 73/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0050 - val_loss: 0.0037
Epoch 74/1000
143/143 [==============================] - ETA: 0s - loss: 0.005 - 0s 133us/step - loss: 0.0050 - val_loss: 0.0063
Epoch 75/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0053 - val_loss: 0.0044
Epoch 76/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0048 - val_loss: 0.0051
Epoch 77/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0048 - val_loss: 0.0044
Epoch 78/1000
143/143 [======================

143/143 [==============================] - 0s 111us/step - loss: 0.0044 - val_loss: 0.0051
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0039
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0050
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0038 - val_loss: 0.0049
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0043
Epoch 151/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0038 - val_loss: 0.0052
Epoch 152/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0035 - val_loss: 0.0040
Epoch 153/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0033 - val_loss: 0.0054
Epoch 154/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0039 - val_loss: 0.0053
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0030 - val_loss: 0.0043
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0054
Epoch 225/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0029 - val_loss: 0.0050
Epoch 226/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0046
Epoch 227/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0027 - val_loss: 0.0050
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0046
Epoch 229/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0028 - val_loss: 0.0046
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0051
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0044
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0045
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0043
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0051
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0051
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0024 - val_loss: 0.0051
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0049
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0046
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0047
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0061
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0040
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0037
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0065
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0046
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0038
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0044
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 840/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0038
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 843/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0013 - val_loss: 0.0049
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0039
Epoch 845/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0043
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0038
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0039
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0039
Epoch 920/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0015 - val_loss: 0.0038
Epoch 921/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0018 - val_loss: 0.0054
Epoch 922/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0019 - val_loss: 0.0038
Epoch 923/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0039
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0049
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 139us/step - loss: 0.0013 - val_loss: 0.0039
Epoch 994/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 995/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0013 - val_loss: 0.0037
Epoch 996/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 997/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0013 - val_loss: 0.0039
Epoch 998/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 999/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0039
Epoch 1000/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0013 - val_loss: 0.0060
0.005600437521934509


[array([[-8.86414051e-01,  1.32266268e-01, -1.06589937e+00,
          4.00397003e-01,  5.15955202e-02],
        [-1.96940199e-01, -8.16048443e-01,  2.59725302e-01,
         -1.29662350e-01, -1.44067153e-01],
        [ 1.47310555e-01, -9.64490592e-01,  4.53336149e-01,
          2.50594735e-01,  5.14563560e-01],
        [-9.31146368e-02, -7.14909554e-01, -2.95087576e-01,
         -4.96858805e-02,  5.22911549e-01],
        [ 2.57437527e-01, -3.09611052e-01, -3.59715484e-02,
         -7.91981071e-02,  9.79757249e-01],
        [ 2.96865609e-02,  1.47427118e+00, -8.40211153e-01,
         -4.46789265e-01, -7.21016619e-03],
        [ 9.64553207e-02, -2.65305758e-01,  3.88571233e-01,
          1.22244157e-01, -8.33731294e-01],
        [-1.11125760e-01,  2.14378268e-01, -3.01701814e-01,
          1.06715865e-01,  4.27079767e-01],
        [ 4.55073593e-03, -8.37093443e-02, -1.65488869e-02,
         -1.18894996e-02,  1.09087563e+00],
        [-7.22066283e-01,  2.58447956e-02, -5.81416070e-01,
    

In [53]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure2_5th.h5')

In [54]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_3(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_30 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

Epoch 69/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0065 - val_loss: 0.0042
Epoch 70/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0065 - val_loss: 0.0050
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0046
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0050
Epoch 73/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0063 - val_loss: 0.0043
Epoch 74/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0061 - val_loss: 0.0044
Epoch 75/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0062 - val_loss: 0.0044
Epoch 76/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0061 - val_loss: 0.0043
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0063 - val_loss: 0.0044
Epoch 78/1000
143/143 [==============================] 

143/143 [==============================] - 0s 126us/step - loss: 0.0055 - val_loss: 0.0045
Epoch 147/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0042
Epoch 148/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0054 - val_loss: 0.0045
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0053 - val_loss: 0.0039
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0053 - val_loss: 0.0043
Epoch 151/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0052 - val_loss: 0.0043
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0041
Epoch 153/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0052 - val_loss: 0.0045
Epoch 154/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0051 - val_loss: 0.0040
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0044
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0046 - val_loss: 0.0040
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0048 - val_loss: 0.0045
Epoch 226/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0049 - val_loss: 0.0043
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0048 - val_loss: 0.0039
Epoch 228/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0045 - val_loss: 0.0045
Epoch 229/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0038
Epoch 230/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0050 - val_loss: 0.0044
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0045
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0043 - val_loss: 0.0047
Epoch 301/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0042 - val_loss: 0.0037
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0042 - val_loss: 0.0040
Epoch 303/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0043 - val_loss: 0.0039
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0038
Epoch 305/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0043 - val_loss: 0.0040
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0040
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0044 - val_loss: 0.0037
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0045
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0044
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0047
Epoch 764/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0022 - val_loss: 0.0050
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0043
Epoch 766/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0041
Epoch 767/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0059
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0045
Epoch 769/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0061
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0053
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0045
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0040
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0043
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0041
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0046
Epoch 845/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0018 - val_loss: 0.0046
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0044
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0041
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0061
Epoch 918/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0020 - val_loss: 0.0041
Epoch 919/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 0.0039
Epoch 920/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0061
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0042
Epoch 922/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0044
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0059
Epoch 924/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0017 - val_loss: 0.0042
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0046
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0046
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0051
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0051
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 998/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0016 - val_loss: 0.0049
Epoch 999/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0047
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0046
0.010777363553643227


[array([[-2.7279971e-02,  2.9943231e-01,  9.0393430e-01, -5.9731603e-01,
         -8.9487797e-01],
        [ 7.4676394e-01,  9.5974666e-01,  4.1767249e-01, -6.1545318e-01,
          4.0519825e-01],
        [ 1.5307154e-01, -3.4375250e-02, -5.5678999e-01,  1.4221966e-01,
          1.1012456e+00],
        [ 1.4741780e-01, -5.9487408e-01,  3.9456129e-01, -5.5574030e-01,
          4.5671150e-01],
        [ 1.9418897e-01,  3.7098399e-01,  3.1744611e-02,  3.4804888e-02,
          3.0280882e-01],
        [ 8.0029619e-01, -4.4479106e-02,  1.0647094e+00,  2.0831505e-02,
         -9.9530458e-01],
        [-1.6531734e-01,  3.3767572e-01,  4.5795050e-01, -5.6175864e-01,
          2.7772254e-01],
        [-5.0927210e-01, -2.9762965e-01,  3.1207989e-03, -4.9909785e-02,
         -1.0117702e-01],
        [ 1.1878413e+00, -2.9919842e-01,  5.6227803e-01,  1.7936060e-01,
          1.4655854e-01],
        [-1.2192087e+00, -1.0600297e-01, -3.9776513e-01, -1.2437670e+00,
         -5.8558998e-03],
        [ 

In [55]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure3_5th.h5')

In [56]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_4(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_31 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_32 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0044
Epoch 70/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0062 - val_loss: 0.0039
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0065 - val_loss: 0.0039
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0065 - val_loss: 0.0039
Epoch 73/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0061 - val_loss: 0.0041
Epoch 74/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0061 - val_loss: 0.0040
Epoch 75/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0061 - val_loss: 0.0042
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0063 - val_loss: 0.0041
Epoch 77/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0062 - val_loss: 0.0041
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0062 - val_loss: 0.0041
Epoch 147/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0059 - val_loss: 0.0047
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0054 - val_loss: 0.0042
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0054 - val_loss: 0.0043
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0042
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0052 - val_loss: 0.0043
Epoch 152/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0041
Epoch 153/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0038
Epoch 154/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0048 - val_loss: 0.0042
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0039
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 225/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 226/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0037 - val_loss: 0.0036
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0043
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0044
Epoch 229/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0038
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0036 - val_loss: 0.0036
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0053
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0036
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0055
Epoch 302/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0026 - val_loss: 0.0038
Epoch 303/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0031 - val_loss: 0.0037
Epoch 304/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0031 - val_loss: 0.0050
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0045
Epoch 306/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 307/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0035
Epoch 308/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0054
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0056
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 764/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 765/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 766/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 767/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0047
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0045
Epoch 769/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0046
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0058
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0046
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 9.9167e-04 - val_loss: 0.0041
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0043
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0044
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0053
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0045
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0044
Epoch 847/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0011 - val_loss: 0.0047
Epoch 848/1000
143/143 [==============================] -

Epoch 916/1000
143/143 [==============================] - 0s 119us/step - loss: 9.9642e-04 - val_loss: 0.0043
Epoch 917/1000
143/143 [==============================] - 0s 126us/step - loss: 9.7017e-04 - val_loss: 0.0044
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 9.7149e-04 - val_loss: 0.0047
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 920/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 921/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0042
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0047
Epoch 923/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0049
Epoch 924/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 925/1000
143/143 [==========

143/143 [==============================] - 0s 119us/step - loss: 9.6409e-04 - val_loss: 0.0042
Epoch 993/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0042
Epoch 994/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 995/1000
143/143 [==============================] - 0s 121us/step - loss: 9.1579e-04 - val_loss: 0.0047
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0055
Epoch 997/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0044
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 999/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0053
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0046
0.007034643553197384


[array([[-0.91722715,  0.5758587 , -0.48319298,  0.09125376,  0.18881534,
         -0.5500606 ,  0.6568376 , -0.34302747,  1.1065031 ,  0.31543764],
        [-0.22976469,  0.18436275,  0.88150686, -0.15410334,  0.40616852,
         -0.38216847,  0.4086883 ,  0.05774491, -0.24632572, -0.35412735],
        [ 0.04137542, -0.02945466, -0.24242914,  0.1576141 ,  0.31490415,
          0.31700954, -0.08720154, -0.26713595, -0.62284094, -0.85291266],
        [ 0.17443593,  0.69980097,  0.40314028, -0.20724869,  0.24679257,
         -0.27532348,  0.46421412, -0.5459795 ,  0.02271914, -0.5305957 ],
        [-0.7128735 ,  0.4603771 ,  0.7173084 , -0.44174784, -0.02010692,
         -0.07123181, -0.12152233, -0.07929222, -0.11691898, -0.01606906],
        [ 0.23514627,  0.24979448,  0.8008222 , -0.45207635,  0.2740736 ,
         -0.60224795,  0.13710837, -0.1515329 , -0.1866573 ,  1.0521652 ],
        [ 0.29991263,  0.05605277, -0.53563833,  0.13459536,  0.57570606,
          0.15409581,  0.6616179

In [57]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure4_5th.h5')

In [58]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_5(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_33 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_34 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0070 - val_loss: 0.0045
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0049
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0073 - val_loss: 0.0056
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0077 - val_loss: 0.0078
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0068 - val_loss: 0.0051
Epoch 74/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0064 - val_loss: 0.0058
Epoch 75/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0044
Epoch 76/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0057 - val_loss: 0.0056
Epoch 77/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0063 - val_loss: 0.0043
Epoch 78/1000
143/143 [==============================] - 0s 133us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0045 - val_loss: 0.0045
Epoch 147/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0042 - val_loss: 0.0049
Epoch 148/1000
143/143 [==============================] - ETA: 0s - loss: 0.005 - 0s 126us/step - loss: 0.0041 - val_loss: 0.0050
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0047 - val_loss: 0.0048
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0047
Epoch 151/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0044 - val_loss: 0.0049
Epoch 152/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0045
Epoch 153/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0039
Epoch 154/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0042 - val_loss: 0.0040
Epoch 155/1000
143/143 [=============

143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0049
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0042
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0040
Epoch 226/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0054
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0036 - val_loss: 0.0052
Epoch 228/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0032 - val_loss: 0.0044
Epoch 229/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0033 - val_loss: 0.0037
Epoch 230/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0033 - val_loss: 0.0044
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0030 - val_loss: 0.0045
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 0.0041
Epoch 301/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0051
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0055
Epoch 303/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0044
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0043
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0038
Epoch 306/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0041
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 308/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0044
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 167us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 763/1000
143/143 [==============================] - 0s 188us/step - loss: 0.0013 - val_loss: 0.0039
Epoch 764/1000
143/143 [==============================] - 0s 174us/step - loss: 0.0013 - val_loss: 0.0038
Epoch 765/1000
143/143 [==============================] - 0s 153us/step - loss: 0.0012 - val_loss: 0.0034
Epoch 766/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0012 - val_loss: 0.0037
Epoch 767/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0035
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0036
Epoch 769/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0037
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0038
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 133us/step - loss: 0.0010 - val_loss: 0.0036
Epoch 840/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0011 - val_loss: 0.0037
Epoch 841/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0011 - val_loss: 0.0037
Epoch 842/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 843/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0016 - val_loss: 0.0038
Epoch 844/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0043
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0037
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0037
Epoch 917/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0010 - val_loss: 0.0036
Epoch 918/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0035
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0039
Epoch 920/1000
143/143 [==============================] - 0s 118us/step - loss: 9.8936e-04 - val_loss: 0.0039
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0036
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0038
Epoch 923/1000
143/143 [==============================] - 0s 125us/step - loss: 9.8182e-04 - val_loss: 0.0036
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0010 - val_loss: 0.0037
Epoch 925/1000
143/143 [=============================

143/143 [==============================] - 0s 126us/step - loss: 9.3158e-04 - val_loss: 0.0037
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 9.8537e-04 - val_loss: 0.0038
Epoch 995/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0011 - val_loss: 0.0036
Epoch 996/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0038
Epoch 998/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0037
Epoch 999/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0039
Epoch 1000/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0036
0.010381639935076237


[array([[-4.29963291e-01,  6.97986409e-02,  3.18107814e-01,
         -1.08309366e-01, -1.01609623e+00,  2.52303839e-01,
          9.79532301e-01, -6.45831823e-01,  4.06560242e-01,
          4.37312245e-01],
        [ 3.57357234e-01,  2.85633445e-01,  4.09653515e-01,
         -1.03565884e+00,  5.35709321e-01,  8.43365669e-01,
          1.93874687e-02, -2.84766018e-01,  9.46947560e-02,
         -2.38238826e-01],
        [ 2.53757000e-01, -1.04769789e-01, -5.15147485e-02,
          4.35683936e-01,  6.39678121e-01,  2.06354886e-01,
         -3.54564250e-01, -7.32497498e-02,  3.95391017e-01,
          1.14027068e-01],
        [ 4.07543153e-01, -1.18131131e-01,  1.56310606e+00,
          4.61439699e-01,  1.17608897e-01,  6.46215677e-01,
          3.94269168e-01, -3.24352942e-02, -1.07234858e-01,
          8.60613137e-02],
        [ 5.17353833e-01,  3.28013927e-01, -2.27990106e-01,
         -7.66262293e-01,  1.09685101e-01,  1.07119095e+00,
          3.97755176e-01, -4.59859163e-01,  1.144884

In [59]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure5_5th.h5')

In [60]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_6(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_35 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_36 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 16        
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0046
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0045
Epoch 71/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0053 - val_loss: 0.0043
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0052
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0053 - val_loss: 0.0045
Epoch 74/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0051 - val_loss: 0.0049
Epoch 75/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0050 - val_loss: 0.0043
Epoch 76/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0050 - val_loss: 0.0047
Epoch 77/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0051 - val_loss: 0.0052
Epoch 78/1000
143/143 [==============================] - 0s 117us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0038
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0040
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0043
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0052
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0051
Epoch 151/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0047
Epoch 152/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0036 - val_loss: 0.0039
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0054
Epoch 154/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0043
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0068
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0049 - val_loss: 0.0046
Epoch 225/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0036 - val_loss: 0.0076
Epoch 226/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0053
Epoch 227/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0030 - val_loss: 0.0041
Epoch 228/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0029 - val_loss: 0.0042
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0045
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0041
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0019 - val_loss: 0.0049
Epoch 301/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0019 - val_loss: 0.0056
Epoch 302/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0019 - val_loss: 0.0041
Epoch 303/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0020 - val_loss: 0.0040
Epoch 304/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0045
Epoch 305/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0039 - val_loss: 0.0071
Epoch 306/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0033 - val_loss: 0.0060
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0036
Epoch 308/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0048
Epoch 309/1000
143/143 [==============================] - 0s 

...

Epoch 762/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0055
Epoch 763/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0054
Epoch 764/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0043
Epoch 765/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0044
Epoch 766/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 767/1000
143/143 [==============================] - 0s 126us/step - loss: 9.7710e-04 - val_loss: 0.0042
Epoch 768/1000
143/143 [==============================] - 0s 126us/step - loss: 9.8534e-04 - val_loss: 0.0043
Epoch 769/1000
143/143 [==============================] - 0s 126us/step - loss: 9.2368e-04 - val_loss: 0.0046
Epoch 770/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 771/1000
143/143 [==========

143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0045
Epoch 839/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0039
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 9.2307e-04 - val_loss: 0.0058
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 9.7252e-04 - val_loss: 0.0047
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 9.1352e-04 - val_loss: 0.0046
Epoch 844/1000
143/143 [==============================] - 0s 126us/step - loss: 8.8809e-04 - val_loss: 0.0045
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 8.4954e-04 - val_loss: 0.0047
Epoch 846/1000
143/143 [==============================] - 0s 125us/step - loss: 8.5914e-04 - val_loss: 0.0049
Epoch 847/1000
143/143 [=============

Epoch 915/1000
143/143 [==============================] - 0s 146us/step - loss: 7.9767e-04 - val_loss: 0.0042
Epoch 916/1000
143/143 [==============================] - 0s 146us/step - loss: 9.1749e-04 - val_loss: 0.0048
Epoch 917/1000
143/143 [==============================] - 0s 133us/step - loss: 8.9808e-04 - val_loss: 0.0049
Epoch 918/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0011 - val_loss: 0.0044
Epoch 919/1000
143/143 [==============================] - 0s 139us/step - loss: 9.1341e-04 - val_loss: 0.0046
Epoch 920/1000
143/143 [==============================] - 0s 139us/step - loss: 9.0932e-04 - val_loss: 0.0048
Epoch 921/1000
143/143 [==============================] - 0s 133us/step - loss: 9.9082e-04 - val_loss: 0.0050
Epoch 922/1000
143/143 [==============================] - 0s 133us/step - loss: 9.5628e-04 - val_loss: 0.0050
Epoch 923/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0049
Epoch 924/1000
143

143/143 [==============================] - 0s 146us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 991/1000
143/143 [==============================] - 0s 160us/step - loss: 0.0010 - val_loss: 0.0045
Epoch 992/1000
143/143 [==============================] - 0s 181us/step - loss: 0.0014 - val_loss: 0.0047
Epoch 993/1000
143/143 [==============================] - 0s 160us/step - loss: 0.0014 - val_loss: 0.0058
Epoch 994/1000
143/143 [==============================] - 0s 188us/step - loss: 0.0015 - val_loss: 0.0052
Epoch 995/1000
143/143 [==============================] - 0s 160us/step - loss: 0.0015 - val_loss: 0.0058
Epoch 996/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0041
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 998/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0051
Epoch 999/1000
143/143 [==============================] - 0s 

[array([[ 3.60642076e-01, -7.80959606e-01, -2.82180756e-01,
         -8.66913617e-01,  3.05884838e-01,  7.65535891e-01,
          2.25269586e-01, -1.02888608e+00, -2.11435959e-01,
         -5.31107664e-01],
        [ 8.24444711e-01, -1.10601828e-01,  1.70715228e-01,
          5.20451725e-01,  5.00187695e-01,  4.27367464e-02,
          5.85199594e-01,  6.08285032e-02, -1.57977313e-01,
          5.06036818e-01],
        [-1.54866293e-01, -5.27568758e-01,  4.11440164e-01,
          6.59774184e-01, -6.28085583e-02, -1.51573643e-01,
         -6.27428368e-02,  7.08883226e-01, -3.03330421e-01,
          9.48388278e-02],
        [-8.49251866e-01, -5.45350134e-01, -9.03452218e-01,
         -4.91938561e-01, -1.85577542e-01,  3.81667674e-01,
          2.90073723e-01,  1.23370981e+00, -4.68990773e-01,
         -6.91534430e-02],
        [-6.48926497e-02,  5.36015213e-01,  7.14949369e-01,
         -4.50780280e-02,  1.44578844e-01,  2.92833447e-02,
         -2.90979296e-01,  1.61800355e-01, -2.344329

In [61]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure6_5th.h5')

In [62]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_1(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_37 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_38 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 6         
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 36.15

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0272 - val_loss: 0.0229
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0272 - val_loss: 0.0223
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0262 - val_loss: 0.0208
Epoch 74/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0244 - val_loss: 0.0189
Epoch 75/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0221 - val_loss: 0.0168
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0199 - val_loss: 0.0148
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0180 - val_loss: 0.0131
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0167 - val_loss: 0.0118
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0161 - val_loss: 0.0109
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0066 - val_loss: 0.0036
Epoch 152/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0065 - val_loss: 0.0036
Epoch 153/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0065 - val_loss: 0.0035
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0064 - val_loss: 0.0035
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0064 - val_loss: 0.0035
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0063 - val_loss: 0.0034
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0063 - val_loss: 0.0034
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0062 - val_loss: 0.0034
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0062 - val_loss: 0.0033
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0020
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0038 - val_loss: 0.0020
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0020
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0019
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0019
Epoch 235/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0038 - val_loss: 0.0019
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0019
Epoch 237/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0037 - val_loss: 0.0019
Epoch 238/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0037 - val_loss: 0.0019
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 0.0013
Epoch 310/1000
32/32 [==============================] - 0s 249us/step - loss: 0.0026 - val_loss: 0.0013
Epoch 311/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 0.0013
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0013
Epoch 313/1000
32/32 [==============================] - 0s 249us/step - loss: 0.0026 - val_loss: 0.0013
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0013
Epoch 315/1000
32/32 [==============================] - 0s 280us/step - loss: 0.0025 - val_loss: 0.0013
Epoch 316/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0025 - val_loss: 0.0013
Epoch 317/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0025 - val_loss: 0.0013
Epoch 318/1000
32/32 [==============================] - 0s 187us

...

Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4038e-04 - val_loss: 4.4764e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3879e-04 - val_loss: 4.4690e-04
Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3721e-04 - val_loss: 4.4618e-04
Epoch 767/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3563e-04 - val_loss: 4.4544e-04
Epoch 768/1000
32/32 [==============================] - 0s 187us/step - loss: 7.3405e-04 - val_loss: 4.4471e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3249e-04 - val_loss: 4.4398e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3092e-04 - val_loss: 4.4328e-04
Epoch 771/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2936e-04 - val_loss: 4.4255e-04
Epoch 772/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2780e-04 - val_loss: 4.4

Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3383e-04 - val_loss: 3.9746e-04
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3253e-04 - val_loss: 3.9682e-04
Epoch 840/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3123e-04 - val_loss: 3.9620e-04
Epoch 841/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2993e-04 - val_loss: 3.9557e-04
Epoch 842/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2865e-04 - val_loss: 3.9495e-04
Epoch 843/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2736e-04 - val_loss: 3.9433e-04
Epoch 844/1000
32/32 [==============================] - 0s 187us/step - loss: 6.2607e-04 - val_loss: 3.9371e-04
Epoch 845/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2479e-04 - val_loss: 3.9308e-04
Epoch 846/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2351e-04 - val_loss: 3.9

Epoch 912/1000
32/32 [==============================] - 0s 187us/step - loss: 5.4590e-04 - val_loss: 3.5374e-04
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4482e-04 - val_loss: 3.5317e-04
Epoch 914/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4374e-04 - val_loss: 3.5264e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4267e-04 - val_loss: 3.5207e-04
Epoch 916/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4160e-04 - val_loss: 3.5153e-04
Epoch 917/1000
32/32 [==============================] - 0s 155us/step - loss: 5.4053e-04 - val_loss: 3.5098e-04
Epoch 918/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3946e-04 - val_loss: 3.5042e-04
Epoch 919/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3840e-04 - val_loss: 3.4988e-04
Epoch 920/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3733e-04 - val_loss: 3.4

Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7257e-04 - val_loss: 3.1520e-04
Epoch 987/1000
32/32 [==============================] - 0s 218us/step - loss: 4.7166e-04 - val_loss: 3.1471e-04
Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7076e-04 - val_loss: 3.1423e-04
Epoch 989/1000
32/32 [==============================] - 0s 187us/step - loss: 4.6986e-04 - val_loss: 3.1374e-04
Epoch 990/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6896e-04 - val_loss: 3.1326e-04
Epoch 991/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6806e-04 - val_loss: 3.1275e-04
Epoch 992/1000
32/32 [==============================] - 0s 187us/step - loss: 4.6717e-04 - val_loss: 3.1228e-04
Epoch 993/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6628e-04 - val_loss: 3.1179e-04
Epoch 994/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6539e-04 - val_loss: 3.1

[array([[-0.00643784, -0.67432886, -0.69756997, -0.01782282,  0.20315535],
        [-0.18590218,  0.6707559 ,  0.6489374 , -0.14127694,  0.3411386 ],
        [-0.1780206 ,  0.9059355 , -0.670769  , -0.33804742,  1.4364766 ]],
       dtype=float32),
 array([-0.65187085, -0.52265525,  0.5999584 ,  0.58907104,  0.45946464],
       dtype=float32),
 array([[ 0.05702494,  0.8888617 , -0.49676722,  0.6230014 , -0.2927776 ],
        [ 0.3184487 ,  0.5937209 ,  0.58160615,  0.8651863 ,  0.24036802],
        [-0.15816599, -0.45121938,  0.14499214, -0.8854137 ,  0.6428694 ],
        [-0.75713044,  0.29873237, -0.9027046 , -0.75456864,  0.7719495 ],
        [ 0.24106322, -0.09745515, -0.82654244, -0.79251784,  0.39338687]],
       dtype=float32),
 array([ 0.61430645, -0.89365065, -0.8551208 , -0.8896346 ,  0.8596787 ],
       dtype=float32),
 array([[ 0.14992635],
        [-0.7578798 ],
        [-0.6890243 ],
        [-0.7817559 ],
        [ 0.63425153]], dtype=float32),
 array([0.97055846], dtype

In [63]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure1_5th.h5')

In [64]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_2(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_39 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_40 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 36.53

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0146 - val_loss: 0.0095
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0115 - val_loss: 0.0063
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0090 - val_loss: 0.0040
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0076 - val_loss: 0.0029
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0072 - val_loss: 0.0028
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0076 - val_loss: 0.0033
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0085 - val_loss: 0.0042
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0094 - val_loss: 0.0050
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0099 - val_loss: 0.0054
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 7.4949e-04
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 7.5896e-04
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 7.7194e-04
Epoch 152/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0020 - val_loss: 7.8797e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 8.0643e-04
Epoch 154/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0020 - val_loss: 8.2654e-04
Epoch 155/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0019 - val_loss: 8.4749e-04
Epoch 156/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0019 - val_loss: 8.6837e-04
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 8.8836e-04
Epoch 158/1000
32/32 [======

Epoch 228/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0013 - val_loss: 0.0017
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 234/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0017
Epoch 237/1000
32/32 [==============================] - 0s 188us

Epoch 307/1000
32/32 [==============================] - 0s 187us/step - loss: 9.9753e-04 - val_loss: 0.0017
Epoch 308/1000
32/32 [==============================] - 0s 187us/step - loss: 9.9492e-04 - val_loss: 0.0017
Epoch 309/1000
32/32 [==============================] - 0s 187us/step - loss: 9.9232e-04 - val_loss: 0.0017
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8971e-04 - val_loss: 0.0017
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8713e-04 - val_loss: 0.0017
Epoch 312/1000
32/32 [==============================] - 0s 157us/step - loss: 9.8455e-04 - val_loss: 0.0017
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8197e-04 - val_loss: 0.0017
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 9.7941e-04 - val_loss: 0.0017
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 9.7685e-04 - val_loss: 0.0017
Epoch 316/1000
32/32 [======

...

Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0604e-04 - val_loss: 4.8052e-04
Epoch 757/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0526e-04 - val_loss: 4.7929e-04
Epoch 758/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0448e-04 - val_loss: 4.7807e-04
Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0370e-04 - val_loss: 4.7686e-04
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0292e-04 - val_loss: 4.7564e-04
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0214e-04 - val_loss: 4.7443e-04
Epoch 762/1000
32/32 [==============================] - 0s 157us/step - loss: 3.0137e-04 - val_loss: 4.7322e-04
Epoch 763/1000
32/32 [==============================] - 0s 186us/step - loss: 3.0060e-04 - val_loss: 4.7201e-04
Epoch 764/1000
32/32 [==============================] - 0s 187us/step - loss: 2.9983e-04 - val_loss: 4.7

Epoch 830/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5360e-04 - val_loss: 4.0081e-04
Epoch 831/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5296e-04 - val_loss: 3.9987e-04
Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5232e-04 - val_loss: 3.9893e-04
Epoch 833/1000
32/32 [==============================] - 0s 142us/step - loss: 2.5169e-04 - val_loss: 3.9800e-04
Epoch 834/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5105e-04 - val_loss: 3.9708e-04
Epoch 835/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5043e-04 - val_loss: 3.9615e-04
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4980e-04 - val_loss: 3.9523e-04
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4918e-04 - val_loss: 3.9432e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4855e-04 - val_loss: 3.9

Epoch 904/1000
32/32 [==============================] - 0s 154us/step - loss: 2.1106e-04 - val_loss: 3.3990e-04
Epoch 905/1000
32/32 [==============================] - 0s 312us/step - loss: 2.1055e-04 - val_loss: 3.3918e-04
Epoch 906/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1003e-04 - val_loss: 3.3848e-04
Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0953e-04 - val_loss: 3.3775e-04
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0901e-04 - val_loss: 3.3705e-04
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0850e-04 - val_loss: 3.3634e-04
Epoch 910/1000
32/32 [==============================] - 0s 187us/step - loss: 2.0799e-04 - val_loss: 3.3563e-04
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0749e-04 - val_loss: 3.3492e-04
Epoch 912/1000
32/32 [==============================] - 0s 187us/step - loss: 2.0698e-04 - val_loss: 3.3

Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7667e-04 - val_loss: 2.9293e-04
Epoch 979/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7625e-04 - val_loss: 2.9238e-04
Epoch 980/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7584e-04 - val_loss: 2.9183e-04
Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7543e-04 - val_loss: 2.9127e-04
Epoch 982/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7502e-04 - val_loss: 2.9071e-04
Epoch 983/1000
32/32 [==============================] - 0s 186us/step - loss: 1.7460e-04 - val_loss: 2.9016e-04
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7419e-04 - val_loss: 2.8960e-04
Epoch 985/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7378e-04 - val_loss: 2.8907e-04
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7337e-04 - val_loss: 2.8

[array([[-0.54169637,  0.34929857,  0.8768658 ,  0.52372444, -0.49833632],
        [ 1.140828  ,  0.03867984, -0.19162105, -0.5043923 , -0.2447526 ],
        [-0.5315727 , -0.35692093,  0.36236218, -0.8627294 , -0.9970089 ]],
       dtype=float32),
 array([ 0.64209014, -0.23871034, -0.6807144 ,  0.5031508 , -0.3620726 ],
       dtype=float32),
 array([[-0.5829349 ,  0.4226227 ,  0.43882722,  0.11619586,  0.516009  ,
          0.25665694, -0.34029523,  0.57242346,  0.03887531, -0.12503229],
        [ 0.29511276,  0.1649536 , -0.04252048,  0.28458565,  0.46312296,
         -0.13703355,  0.12020335, -0.48242283,  0.3994536 , -0.15723334],
        [-0.6967453 , -0.19629787,  0.33185515, -0.16314606,  0.13715091,
         -0.01736727,  0.25997677, -0.3552291 , -0.36874533, -0.22631767],
        [ 0.46257073,  0.62157387, -0.17002627, -0.15960087,  0.14568098,
         -0.1596013 , -0.35578078,  0.4262544 ,  0.30941567,  0.58108246],
        [-0.6150803 , -0.7495975 ,  0.23389503, -0.0480860

In [65]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure2_5th.h5')

In [66]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_3(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_62 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_42 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 16        
Total params: 126
Trainable params: 126
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0033
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0046
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0052
Epoch 75/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0041 - val_loss: 0.0055
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0044 - val_loss: 0.0053
Epoch 77/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0042 - val_loss: 0.0047
Epoch 78/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 79/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0029 - val_loss: 0.0033
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6768e-04 - val_loss: 8.8388e-04
Epoch 150/1000
32/32 [==============================] - 0s 157us/step - loss: 5.6128e-04 - val_loss: 8.7789e-04
Epoch 151/1000
32/32 [==============================] - 0s 155us/step - loss: 5.5507e-04 - val_loss: 8.7053e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4892e-04 - val_loss: 8.6163e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 5.4278e-04 - val_loss: 8.5131e-04
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 5.3660e-04 - val_loss: 8.3981e-04
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3043e-04 - val_loss: 8.2757e-04
Epoch 156/1000
32/32 [==============================] - 0s 187us/step - loss: 5.2432e-04 - val_loss: 8.1502e-04
Epoch 157/1000
32/32 [==============================] - 0s 187us/step - loss: 5.1834e-04 - val_loss: 8.0

Epoch 223/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7880e-04 - val_loss: 4.1549e-04
Epoch 224/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7656e-04 - val_loss: 4.1203e-04
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7434e-04 - val_loss: 4.0862e-04
Epoch 226/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7215e-04 - val_loss: 4.0525e-04
Epoch 227/1000
32/32 [==============================] - 0s 187us/step - loss: 2.6999e-04 - val_loss: 4.0188e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6784e-04 - val_loss: 3.9853e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6573e-04 - val_loss: 3.9522e-04
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6363e-04 - val_loss: 3.9191e-04
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 2.6157e-04 - val_loss: 3.8

Epoch 297/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6374e-04 - val_loss: 2.3772e-04
Epoch 298/1000
32/32 [==============================] - 0s 187us/step - loss: 1.6269e-04 - val_loss: 2.3613e-04
Epoch 299/1000
32/32 [==============================] - 0s 187us/step - loss: 1.6164e-04 - val_loss: 2.3455e-04
Epoch 300/1000
32/32 [==============================] - 0s 218us/step - loss: 1.6061e-04 - val_loss: 2.3298e-04
Epoch 301/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5958e-04 - val_loss: 2.3143e-04
Epoch 302/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5857e-04 - val_loss: 2.2988e-04
Epoch 303/1000
32/32 [==============================] - 0s 155us/step - loss: 1.5756e-04 - val_loss: 2.2838e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5656e-04 - val_loss: 2.2687e-04
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5557e-04 - val_loss: 2.2

...

Epoch 741/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9463e-05 - val_loss: 2.6800e-05
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9384e-05 - val_loss: 2.6688e-05
Epoch 743/1000
32/32 [==============================] - 0s 125us/step - loss: 1.9305e-05 - val_loss: 2.6577e-05
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9225e-05 - val_loss: 2.6468e-05
Epoch 745/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9147e-05 - val_loss: 2.6358e-05
Epoch 746/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9068e-05 - val_loss: 2.6250e-05
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8991e-05 - val_loss: 2.6139e-05
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8912e-05 - val_loss: 2.6034e-05
Epoch 749/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8836e-05 - val_loss: 2.5

Epoch 815/1000
32/32 [==============================] - 0s 187us/step - loss: 1.4382e-05 - val_loss: 1.9744e-05
Epoch 816/1000
32/32 [==============================] - 0s 155us/step - loss: 1.4323e-05 - val_loss: 1.9664e-05
Epoch 817/1000
32/32 [==============================] - 0s 126us/step - loss: 1.4266e-05 - val_loss: 1.9583e-05
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4206e-05 - val_loss: 1.9502e-05
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4149e-05 - val_loss: 1.9421e-05
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4092e-05 - val_loss: 1.9343e-05
Epoch 821/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4035e-05 - val_loss: 1.9264e-05
Epoch 822/1000
32/32 [==============================] - 0s 156us/step - loss: 1.3979e-05 - val_loss: 1.9184e-05
Epoch 823/1000
32/32 [==============================] - 0s 125us/step - loss: 1.3921e-05 - val_loss: 1.9

Epoch 889/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0637e-05 - val_loss: 1.4572e-05
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0593e-05 - val_loss: 1.4509e-05
Epoch 891/1000
32/32 [==============================] - 0s 186us/step - loss: 1.0550e-05 - val_loss: 1.4450e-05
Epoch 892/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0507e-05 - val_loss: 1.4390e-05
Epoch 893/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0463e-05 - val_loss: 1.4331e-05
Epoch 894/1000
32/32 [==============================] - 0s 125us/step - loss: 1.0422e-05 - val_loss: 1.4272e-05
Epoch 895/1000
32/32 [==============================] - 0s 155us/step - loss: 1.0379e-05 - val_loss: 1.4213e-05
Epoch 896/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0337e-05 - val_loss: 1.4153e-05
Epoch 897/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0294e-05 - val_loss: 1.4

Epoch 963/1000
32/32 [==============================] - 0s 156us/step - loss: 7.8529e-06 - val_loss: 1.0736e-05
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 7.8192e-06 - val_loss: 1.0695e-05
Epoch 965/1000
32/32 [==============================] - 0s 187us/step - loss: 7.7880e-06 - val_loss: 1.0647e-05
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7553e-06 - val_loss: 1.0605e-05
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7231e-06 - val_loss: 1.0560e-05
Epoch 968/1000
32/32 [==============================] - 0s 187us/step - loss: 7.6921e-06 - val_loss: 1.0515e-05
Epoch 969/1000
32/32 [==============================] - 0s 125us/step - loss: 7.6597e-06 - val_loss: 1.0472e-05
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 7.6292e-06 - val_loss: 1.0431e-05
Epoch 971/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5969e-06 - val_loss: 1.0

[array([[ 0.7003244 ,  1.3716525 , -0.10923705, -0.19709598, -0.47631347],
        [-0.21304703, -0.26567638, -0.4049768 ,  0.6179817 ,  0.8688171 ],
        [-0.1713454 , -0.41981235,  0.8031276 , -0.90102935, -1.0292379 ]],
       dtype=float32),
 array([ 0.14315966, -0.15401782, -0.4217508 ,  0.7311326 , -0.56353766],
       dtype=float32),
 array([[-0.33576006,  0.05490396,  0.29599604,  0.17336951, -0.11966818,
         -0.31975496,  0.1966273 , -0.18240315, -0.2947267 ,  0.2086887 ,
         -0.04697833, -0.01192727,  0.5100307 , -0.10733802,  0.381238  ],
        [ 0.40976143, -0.01556473,  0.28002492,  0.24071418, -0.03901509,
         -0.14970142,  0.00382109, -0.27926257, -0.16640183,  0.42351407,
          0.32319412,  0.26947573,  0.34322205,  0.2516954 , -0.44240528],
        [-0.2537775 , -0.3261055 ,  0.32144028, -0.53254855,  0.19507056,
          0.06046795,  0.39527306, -0.4809832 ,  0.23045105,  0.31690848,
         -0.4140889 , -0.480011  , -0.15833531,  0.16021901,

In [67]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure3_5th.h5')

In [68]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_4(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_43 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_44 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 6         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0126 - val_loss: 0.0196
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0118 - val_loss: 0.0176
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0115 - val_loss: 0.0161
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0115 - val_loss: 0.0150
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0116 - val_loss: 0.0145
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0118 - val_loss: 0.0144
Epoch 77/1000
32/32 [==============================] - 0s 188us/step - loss: 0.0119 - val_loss: 0.0145
Epoch 78/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0120 - val_loss: 0.0145
Epoch 79/1000
32/32 [==============================] - 0s 181us/step - loss: 0.0118 - val_loss: 0.0140
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0044 - val_loss: 0.0031
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0044 - val_loss: 0.0031
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0044 - val_loss: 0.0030
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0030
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0030
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0042 - val_loss: 0.0029
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0042 - val_loss: 0.0029
Epoch 158/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0042 - val_loss: 0.0029
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0041 - val_loss: 0.0028
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0011
Epoch 231/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0024 - val_loss: 0.0011
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0024 - val_loss: 0.0011
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0011
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0011
Epoch 235/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0023 - val_loss: 0.0011
Epoch 236/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0011
Epoch 237/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0011
Epoch 238/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0023 - val_loss: 0.0010
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 307/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0014 - val_loss: 4.9162e-04
Epoch 308/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0014 - val_loss: 4.8654e-04
Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 4.8154e-04
Epoch 310/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0014 - val_loss: 4.7657e-04
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 4.7168e-04
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 4.6684e-04
Epoch 313/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 4.6205e-04
Epoch 314/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0014 - val_loss: 4.5732e-04
Epoch 315/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0014 - val_loss: 4.5266e-04
Epoch 316/1000
32/32 [======

...

Epoch 753/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5285e-04 - val_loss: 2.2603e-05
Epoch 754/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5220e-04 - val_loss: 2.2523e-05
Epoch 755/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5155e-04 - val_loss: 2.2444e-05
Epoch 756/1000
32/32 [==============================] - 0s 125us/step - loss: 1.5091e-04 - val_loss: 2.2364e-05
Epoch 757/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5027e-04 - val_loss: 2.2282e-05
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4963e-04 - val_loss: 2.2204e-05
Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4900e-04 - val_loss: 2.2127e-05
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4836e-04 - val_loss: 2.2052e-05
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4774e-04 - val_loss: 2.1

Epoch 827/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1204e-04 - val_loss: 1.7941e-05
Epoch 828/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1158e-04 - val_loss: 1.7892e-05
Epoch 829/1000
32/32 [==============================] - 0s 126us/step - loss: 1.1112e-04 - val_loss: 1.7844e-05
Epoch 830/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1066e-04 - val_loss: 1.7792e-05
Epoch 831/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1020e-04 - val_loss: 1.7747e-05
Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0975e-04 - val_loss: 1.7699e-05
Epoch 833/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0930e-04 - val_loss: 1.7651e-05
Epoch 834/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0885e-04 - val_loss: 1.7602e-05
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0840e-04 - val_loss: 1.7

Epoch 901/1000
32/32 [==============================] - 0s 218us/step - loss: 8.2848e-05 - val_loss: 1.5060e-05
Epoch 902/1000
32/32 [==============================] - 0s 156us/step - loss: 8.2518e-05 - val_loss: 1.5029e-05
Epoch 903/1000
32/32 [==============================] - 0s 155us/step - loss: 8.2186e-05 - val_loss: 1.4999e-05
Epoch 904/1000
32/32 [==============================] - 0s 186us/step - loss: 8.1856e-05 - val_loss: 1.4967e-05
Epoch 905/1000
32/32 [==============================] - 0s 187us/step - loss: 8.1528e-05 - val_loss: 1.4938e-05
Epoch 906/1000
32/32 [==============================] - 0s 187us/step - loss: 8.1200e-05 - val_loss: 1.4909e-05
Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0874e-05 - val_loss: 1.4878e-05
Epoch 908/1000
32/32 [==============================] - 0s 157us/step - loss: 8.0552e-05 - val_loss: 1.4850e-05
Epoch 909/1000
32/32 [==============================] - 0s 188us/step - loss: 8.0228e-05 - val_loss: 1.4

Epoch 975/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1767e-05 - val_loss: 1.3215e-05
Epoch 976/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1528e-05 - val_loss: 1.3195e-05
Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1283e-05 - val_loss: 1.3174e-05
Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1047e-05 - val_loss: 1.3156e-05
Epoch 979/1000
32/32 [==============================] - 0s 187us/step - loss: 6.0809e-05 - val_loss: 1.3135e-05
Epoch 980/1000
32/32 [==============================] - 0s 125us/step - loss: 6.0573e-05 - val_loss: 1.3118e-05
Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0335e-05 - val_loss: 1.3098e-05
Epoch 982/1000
32/32 [==============================] - 0s 218us/step - loss: 6.0100e-05 - val_loss: 1.3079e-05
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9864e-05 - val_loss: 1.3

[array([[ 0.94610345,  0.41327706,  0.7720632 , -0.4662407 , -0.98158497,
         -1.0460271 ,  0.82337326, -1.0866001 ,  1.186878  , -0.73338014],
        [ 0.30200973, -0.31318766,  0.39154273,  0.5153439 ,  1.1756387 ,
          0.21598917, -0.4899142 ,  0.24489994, -0.55670434, -0.2375365 ],
        [ 0.4298186 , -0.78177315,  1.788991  , -0.5736554 , -0.10270688,
         -1.1737224 , -1.500764  , -0.43503585, -0.02571201, -0.37731332]],
       dtype=float32),
 array([ 0.49049008,  0.546256  ,  0.55173004,  0.4475778 , -0.37681216,
        -0.4844004 ,  0.7743582 ,  0.6425515 , -0.44952568,  0.04696493],
       dtype=float32),
 array([[-0.05451885,  0.5881471 ,  0.34906703, -0.99869144,  0.64393485],
        [-0.07821044,  0.519115  , -0.2415335 ,  0.26172388,  0.2598864 ],
        [-0.32703453,  0.8575481 , -0.7181925 ,  0.12325826,  0.49079928],
        [-0.10414832,  0.99715537, -0.22172897, -0.5627915 ,  0.28607908],
        [ 0.32812244, -0.11278784, -0.49714422,  0.15351322

In [69]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure4_5th.h5')

In [70]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_5(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_45 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_46 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0097 - val_loss: 0.0033
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0074 - val_loss: 0.0025
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0064 - val_loss: 0.0024
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0066 - val_loss: 0.0029
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0074 - val_loss: 0.0034
Epoch 76/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0083 - val_loss: 0.0036
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0089 - val_loss: 0.0035
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0088 - val_loss: 0.0030
Epoch 79/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0080 - val_loss: 0.0024
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 148/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0012 - val_loss: 3.8205e-04
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.7807e-04
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.7471e-04
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.7170e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.6874e-04
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0011 - val_loss: 3.6557e-04
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.6202e-04
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.5788e-04
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 3.5305e-04
Epoch 157/1000
32/32 [======

Epoch 222/1000
32/32 [==============================] - 0s 155us/step - loss: 6.8483e-04 - val_loss: 1.4581e-04
Epoch 223/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8094e-04 - val_loss: 1.4380e-04
Epoch 224/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7708e-04 - val_loss: 1.4184e-04
Epoch 225/1000
32/32 [==============================] - 0s 125us/step - loss: 6.7327e-04 - val_loss: 1.3992e-04
Epoch 226/1000
32/32 [==============================] - 0s 188us/step - loss: 6.6949e-04 - val_loss: 1.3805e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6574e-04 - val_loss: 1.3619e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6203e-04 - val_loss: 1.3438e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5835e-04 - val_loss: 1.3258e-04
Epoch 230/1000
32/32 [==============================] - 0s 155us/step - loss: 6.5472e-04 - val_loss: 1.3

Epoch 296/1000
32/32 [==============================] - 0s 218us/step - loss: 4.7247e-04 - val_loss: 5.1227e-05
Epoch 297/1000
32/32 [==============================] - 0s 249us/step - loss: 4.7036e-04 - val_loss: 5.0499e-05
Epoch 298/1000
32/32 [==============================] - 0s 187us/step - loss: 4.6828e-04 - val_loss: 4.9775e-05
Epoch 299/1000
32/32 [==============================] - 0s 187us/step - loss: 4.6621e-04 - val_loss: 4.9063e-05
Epoch 300/1000
32/32 [==============================] - 0s 218us/step - loss: 4.6416e-04 - val_loss: 4.8367e-05
Epoch 301/1000
32/32 [==============================] - 0s 218us/step - loss: 4.6212e-04 - val_loss: 4.7680e-05
Epoch 302/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6009e-04 - val_loss: 4.6999e-05
Epoch 303/1000
32/32 [==============================] - 0s 218us/step - loss: 4.5807e-04 - val_loss: 4.6326e-05
Epoch 304/1000
32/32 [==============================] - 0s 218us/step - loss: 4.5607e-04 - val_loss: 4.5

...

Epoch 740/1000
32/32 [==============================] - 0s 155us/step - loss: 1.0028e-04 - val_loss: 1.0092e-06
Epoch 741/1000
32/32 [==============================] - 0s 125us/step - loss: 9.9960e-05 - val_loss: 1.0073e-06
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9643e-05 - val_loss: 1.0053e-06
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9328e-05 - val_loss: 1.0027e-06
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9011e-05 - val_loss: 1.0007e-06
Epoch 745/1000
32/32 [==============================] - 0s 187us/step - loss: 9.8697e-05 - val_loss: 9.9848e-07
Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8385e-05 - val_loss: 9.9607e-07
Epoch 747/1000
32/32 [==============================] - 0s 187us/step - loss: 9.8076e-05 - val_loss: 9.9405e-07
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 9.7762e-05 - val_loss: 9.9

Epoch 814/1000
32/32 [==============================] - 0s 187us/step - loss: 7.9414e-05 - val_loss: 8.5696e-07
Epoch 815/1000
32/32 [==============================] - 0s 187us/step - loss: 7.9168e-05 - val_loss: 8.5526e-07
Epoch 816/1000
32/32 [==============================] - 0s 218us/step - loss: 7.8921e-05 - val_loss: 8.5337e-07
Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 7.8680e-05 - val_loss: 8.5038e-07
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 7.8431e-05 - val_loss: 8.4886e-07
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 7.8188e-05 - val_loss: 8.4683e-07
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7943e-05 - val_loss: 8.4475e-07
Epoch 821/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7703e-05 - val_loss: 8.4316e-07
Epoch 822/1000
32/32 [==============================] - 0s 156us/step - loss: 7.7461e-05 - val_loss: 8.4

Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3244e-05 - val_loss: 7.2125e-07
Epoch 889/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3054e-05 - val_loss: 7.1970e-07
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2863e-05 - val_loss: 7.1810e-07
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2670e-05 - val_loss: 7.1620e-07
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2478e-05 - val_loss: 7.1507e-07
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2291e-05 - val_loss: 7.1339e-07
Epoch 894/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2104e-05 - val_loss: 7.1201e-07
Epoch 895/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1917e-05 - val_loss: 7.1023e-07
Epoch 896/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1732e-05 - val_loss: 7.0

Epoch 962/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0777e-05 - val_loss: 6.0812e-07
Epoch 963/1000
32/32 [==============================] - 0s 125us/step - loss: 5.0632e-05 - val_loss: 6.0708e-07
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0484e-05 - val_loss: 6.0564e-07
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0337e-05 - val_loss: 6.0446e-07
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0193e-05 - val_loss: 6.0278e-07
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0048e-05 - val_loss: 6.0179e-07
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9905e-05 - val_loss: 6.0041e-07
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9761e-05 - val_loss: 5.9855e-07
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 4.9619e-05 - val_loss: 5.9

[array([[ 0.3325261 , -0.844683  ,  0.97657955,  0.4105869 ,  0.5725055 ,
         -0.579156  , -0.8026457 ,  0.7976244 ,  0.6541244 ,  0.8291424 ],
        [ 0.0207253 , -0.20953079,  0.3695538 ,  0.43669868, -0.45147455,
         -0.07348356,  0.42713615,  0.26468226,  0.5323617 , -0.59026766],
        [ 0.6849398 , -0.88367456,  1.0675265 , -0.84761286, -1.1646522 ,
         -1.1898082 , -0.2433673 , -0.15657163,  0.13903186,  0.05548193]],
       dtype=float32),
 array([-0.2768422 , -0.06329912, -0.08821055,  0.35606754,  0.37501922,
        -0.5128035 ,  0.5762416 ,  0.1690545 , -0.50230074,  0.15018025],
       dtype=float32),
 array([[-0.3266373 ,  0.12549192,  0.09959105,  0.0198029 ,  0.11573185,
          0.5700498 , -0.24181782, -0.2895241 , -0.15913023, -0.24308196],
        [ 0.09561577,  0.20467424,  0.19084473,  0.38947076,  0.19508551,
          0.0207237 ,  0.47219488,  0.3960431 , -0.12616189, -0.49262142],
        [ 0.41243988,  0.22956719, -0.4106159 , -0.46400768, 

In [71]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure5_5th.h5')

In [72]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_6(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_47 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_48 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0097 - val_loss: 0.0160
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0102 - val_loss: 0.0165
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0112 - val_loss: 0.0169
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0120 - val_loss: 0.0168
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0120 - val_loss: 0.0163
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0112 - val_loss: 0.0158
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0100 - val_loss: 0.0156
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0089 - val_loss: 0.0160
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0082 - val_loss: 0.0170
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0032 - val_loss: 0.0036
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0031 - val_loss: 0.0036
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0031 - val_loss: 0.0036
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0031 - val_loss: 0.0035
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0031 - val_loss: 0.0035
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0030 - val_loss: 0.0034
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0030 - val_loss: 0.0033
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0030 - val_loss: 0.0033
Epoch 159/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0030 - val_loss: 0.0032
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 231/1000
32/32 [==============================] - 0s 188us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 232/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 236/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 0.0016
Epoch 237/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 0.0016
Epoch 238/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0018 - val_loss: 0.0016
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0013 - val_loss: 0.0011
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0011
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0011
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0011
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0010
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0010
Epoch 315/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0013 - val_loss: 0.0010
Epoch 316/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0013 - val_loss: 0.0010
Epoch 317/1000
32/32 [==============================] - 0s 249us/step - loss: 0.0013 - val_loss: 0.0010
Epoch 318/1000
32/32 [==============================] - 0s 218us

...

Epoch 755/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6982e-04 - val_loss: 3.9307e-04
Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6915e-04 - val_loss: 3.9266e-04
Epoch 757/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6848e-04 - val_loss: 3.9223e-04
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6783e-04 - val_loss: 3.9182e-04
Epoch 759/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6717e-04 - val_loss: 3.9139e-04
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6651e-04 - val_loss: 3.9096e-04
Epoch 761/1000
32/32 [==============================] - 0s 186us/step - loss: 2.6585e-04 - val_loss: 3.9056e-04
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6520e-04 - val_loss: 3.9014e-04
Epoch 763/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6456e-04 - val_loss: 3.8

Epoch 829/1000
32/32 [==============================] - 0s 187us/step - loss: 2.2749e-04 - val_loss: 3.6427e-04
Epoch 830/1000
32/32 [==============================] - 0s 187us/step - loss: 2.2701e-04 - val_loss: 3.6388e-04
Epoch 831/1000
32/32 [==============================] - 0s 187us/step - loss: 2.2653e-04 - val_loss: 3.6353e-04
Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2604e-04 - val_loss: 3.6316e-04
Epoch 833/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2556e-04 - val_loss: 3.6283e-04
Epoch 834/1000
32/32 [==============================] - 0s 187us/step - loss: 2.2509e-04 - val_loss: 3.6248e-04
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2461e-04 - val_loss: 3.6211e-04
Epoch 836/1000
32/32 [==============================] - 0s 157us/step - loss: 2.2414e-04 - val_loss: 3.6176e-04
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 2.2367e-04 - val_loss: 3.6

Epoch 903/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9647e-04 - val_loss: 3.3936e-04
Epoch 904/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9612e-04 - val_loss: 3.3905e-04
Epoch 905/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9576e-04 - val_loss: 3.3875e-04
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9540e-04 - val_loss: 3.3842e-04
Epoch 907/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9504e-04 - val_loss: 3.3812e-04
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9469e-04 - val_loss: 3.3781e-04
Epoch 909/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9434e-04 - val_loss: 3.3750e-04
Epoch 910/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9398e-04 - val_loss: 3.3717e-04
Epoch 911/1000
32/32 [==============================] - 0s 155us/step - loss: 1.9363e-04 - val_loss: 3.3

Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7316e-04 - val_loss: 3.1738e-04
Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7289e-04 - val_loss: 3.1710e-04
Epoch 979/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7262e-04 - val_loss: 3.1682e-04
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7234e-04 - val_loss: 3.1656e-04
Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7207e-04 - val_loss: 3.1627e-04
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7180e-04 - val_loss: 3.1600e-04
Epoch 983/1000
32/32 [==============================] - 0s 155us/step - loss: 1.7153e-04 - val_loss: 3.1571e-04
Epoch 984/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7126e-04 - val_loss: 3.1544e-04
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7099e-04 - val_loss: 3.1

[array([[ 0.02840035,  0.7275514 , -0.06116987,  0.7246465 ,  0.8211368 ,
          0.74275017,  0.03093249,  0.11779617,  0.12036306,  0.4074682 ],
        [ 0.62594575,  0.42679614, -1.0505158 , -0.5349287 ,  0.05530798,
         -0.9914377 ,  1.126905  , -0.15398486, -1.0946863 , -0.24366997],
        [-0.6145034 , -0.09637294, -0.48746818,  0.34285298,  1.0209967 ,
         -0.40930632,  0.5718359 ,  0.41562778,  0.59960085, -0.8076888 ]],
       dtype=float32),
 array([ 0.32418174, -0.40026075,  0.30656138, -0.26804993,  0.5152698 ,
         0.1537482 , -0.02761584,  0.11173016, -0.15709357,  0.2966469 ],
       dtype=float32),
 array([[-0.66298807, -0.00501132, -0.41127336,  0.20148064,  0.3841239 ,
          0.18888931, -0.54989773, -0.36166343,  0.31015155, -0.02591513,
          0.47445273, -0.02532476, -0.5555552 ,  0.12766576, -0.44596863],
        [-0.05212485, -0.2886192 , -0.17973769, -0.1962559 , -0.22687879,
          0.09936856,  0.23724389,  0.4743854 ,  0.4132825 , -

In [73]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure6_5th.h5')